In [ ]:
import gc
import os
import sys
import time
import glob
import json
import pickle
import tempfile
import traceback
from dataclasses import dataclass, asdict
from contextlib import contextmanager
from copy import deepcopy

import cv2
import optuna
import tensorflow as tf
from tensorflow.keras import backend as K

import lightgbm as lgb
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import treelite
import treelite_runtime
from scipy import special
from scipy.optimize import minimize
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, PredefinedSplit
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import LabelEncoder

from typing import Union
from typing import List, Optional

optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
@dataclass
class Config:
    INPUT_DIR = "../../input"
    CACHE_DIR = "../../cache"

    # competition data
    INPUT = f"{INPUT_DIR}/nfl-player-contact-detection"
    
    # pre-required datasets
    CAMARO_DF1_PATH = f'{INPUT_DIR}/nfl-exp048/val_df.csv'
    CAMARO_DF2_PATH = f'{INPUT_DIR}/camaro-exp117/exp117_val_preds.csv'
    KMAT_PATH = f'{INPUT_DIR}/mfl2cnnkmat0221'

    IS_TRAIN = True

    KMAT_CNN_ALL_FRAME = False
    TREELITE_COMPILE= True
    USE_GPU = True  # set False when running without GPU

cfg = Config()

In [ ]:

sys.path.insert(1, cfg.KMAT_PATH)

os.makedirs(cfg.CACHE_DIR, exist_ok=True)

from train_contact_det import NFLContact, view_contact_mask  # この辺バージョンの違いはない？
from train_utils.dataloader import inference_preprocess

kalman smooth系の関数追加。

0219 update

del df["datetime_ngs"] 残しといてもらえると嬉しいのでとりあえずキープ。CNN終了後に消します。

In [ ]:
@contextmanager
def timer(name: str):
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f"[{name}] {elapsed:.3f}sec")

        
    
class LabelEncoders:
    def __init__(self):
        self.encoders = {}
    
    def fit(self, df):
        for c in df:
            if df[c].dtype.name == "object":
                enc = self.encoders.get(c, LabelEncoder())
                enc.fit(df[c])
                self.encoders[c] = enc
                
    def transform(self, df):
        for c in df:
            if c in self.encoders:
                df[c] = self.encoders[c].transform(df[c])
        return df
    
    def fit_one(self, s):
        enc = self.encoders.get(s.name, LabelEncoder())
        enc.fit(s)
        self.encoders[s.name] = enc
        
    def transform_one(self, s):
        if s.name in self.encoders:
            return self.encoders[s.name].transform(s)
        else:
            return s

    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)
                
    def fit_transform_one(self, s):
        self.fit_one(s)
        return self.transform_one(s)

    def patch(self, target: str = "position_2"):
        # nanの時とNoneの時がある。featherでのシリアライズを噛ませているかどうかで変わってしまう。ここでNoneに合わせる。
        try:
            if np.isnan(self.encoders[target].classes_[-1]):
                self.encoders[target].classes_[-1] = None  # nan to None (patch)
        except Exception:
            print(self.encoders[target].classes_[-1])
            print(type(self.encoders[target].classes_[-1]))

    
def cast_player_id(df):
    # RAM消費を減らしたいので、Gを-1に置換して整数で持つ。
    if "nfl_player_id_2" in df.columns:
        df.loc[df["nfl_player_id_2"] == "G", "nfl_player_id_2"] = "-1"
        
    for c in ["nfl_player_id", "nfl_player_id_1", "nfl_player_id_2"]:
        if c in df.columns:
            df[c] = df[c].astype(np.int32)
            
    return df


def reduce_dtype(df):
    for c in df.columns:
        if df[c].dtype.name == "float64":
            df[c] = df[c].astype(np.float32)
    return df


def distance(x1, y1, x2, y2):
    return np.sqrt(np.square(x2 - x1) + np.square(y2 - y1))
    
    
def add_bbox_move_features(df):
    shifts = [1]

    for shift in shifts:
        for view in ["Endzone", "Sideline"]:
            for pl in ["1", "2"]:
                for xy in ["x", "y"]:
                    df[f"bbox_center_{xy}_{view}_{pl}_m{shift}"] = df.groupby(["game_play", "nfl_player_id_1", "nfl_player_id_2"])[f"bbox_center_{xy}_{view}_{pl}"].shift(shift)
                    df[f"bbox_center_{xy}_{view}_{pl}_p{shift}"] = df.groupby(["game_play", "nfl_player_id_1", "nfl_player_id_2"])[f"bbox_center_{xy}_{view}_{pl}"].shift(-shift)

                df[f"bbox_move_{view}_{pl}_m{shift}"] = distance(
                    df[f"bbox_center_x_{view}_{pl}_m{shift}"],
                    df[f"bbox_center_y_{view}_{pl}_m{shift}"],
                    df[f"bbox_center_x_{view}_{pl}"],
                    df[f"bbox_center_y_{view}_{pl}"],
                )
                df[f"bbox_move_{view}_{pl}_p{shift}"] = distance(
                    df[f"bbox_center_x_{view}_{pl}_p{shift}"],
                    df[f"bbox_center_y_{view}_{pl}_p{shift}"],
                    df[f"bbox_center_x_{view}_{pl}"],
                    df[f"bbox_center_y_{view}_{pl}"],
                )

                for xy in ["x", "y"]:
                    del df[f"bbox_center_{xy}_{view}_{pl}_m{shift}"]
                    del df[f"bbox_center_{xy}_{view}_{pl}_p{shift}"]

            df[f"bbox_move_{view}_m{shift}_mean_1"] = df.groupby(["nfl_player_id_1", "game_play"])[f"bbox_move_{view}_1_m{shift}"].transform("mean")
            df[f"bbox_move_{view}_m{shift}_max_1"] = df.groupby(["nfl_player_id_1", "game_play"])[f"bbox_move_{view}_1_m{shift}"].transform("max")

    return df

    
def merge_helmet(df, helmet, meta):
    start_times = meta[["game_play", "start_time"]].drop_duplicates()
    start_times["start_time"] = pd.to_datetime(start_times["start_time"])

    helmet = pd.merge(helmet,
                      start_times,
                      on="game_play",
                      how="left")
    
    # 追加
    helmet['xmin'] = helmet['left']
    helmet['ymin'] = helmet['top']
    helmet['xmax'] = helmet['left'] + helmet['width']
    helmet['ymax'] = helmet['top'] + helmet['height']
    bbox = np.array([helmet['xmin'], 
                     helmet['ymin'], 
                     helmet['xmax'], 
                     helmet['ymax']]).T
    helmet['bbox_center_x'] = (bbox[:, 0] + bbox[:, 2]) / 2
    helmet['bbox_center_y'] = (bbox[:, 1] + bbox[:, 3]) / 2
    
    
    fps=59.94
    helmet["datetime"] = helmet["start_time"] + pd.to_timedelta(helmet["frame"] * (1 / fps), unit="s")
    helmet["datetime"] = pd.to_datetime(helmet["datetime"], utc=True)
    helmet["datetime_ngs"] = pd.DatetimeIndex(helmet["datetime"] + pd.to_timedelta(50, "ms")).floor("100ms").values
    helmet["datetime_ngs"] = pd.to_datetime(helmet["datetime_ngs"], utc=True)
    df["datetime_ngs"] = pd.to_datetime(df["datetime"], utc=True)
    
    # 追加
    feature_cols = ["width", "height",  "bbox_center_x", "bbox_center_y",
                    "bbox_center_x_std", "bbox_center_y_std"]
    helmet_agg = helmet.groupby(["datetime_ngs", "nfl_player_id", "view"]).agg(
        width=pd.NamedAgg("width", "mean"),
        height=pd.NamedAgg("height", "mean"),
        bbox_center_x=pd.NamedAgg("bbox_center_x", "mean"),
        bbox_center_y=pd.NamedAgg("bbox_center_y", "mean"),
        bbox_center_x_std=pd.NamedAgg("bbox_center_x", "std"),
        bbox_center_y_std=pd.NamedAgg("bbox_center_y", "std"),
    ).reset_index()

    for view in ["Sideline", "Endzone"]:
        helmet_ = helmet_agg[helmet_agg["view"]==view].drop("view", axis=1)
    
        helmet_global = helmet[helmet["view"]==view].groupby("datetime_ngs").agg(
        **{
            f"width_{view}_mean": pd.NamedAgg("width", "mean"),
            f"height_{view}_mean": pd.NamedAgg("height", "mean"),
            f"{view}_count": pd.NamedAgg("width", "count"),
        }).reset_index()
        
        helmet_global[f"aspect_{view}_mean"] = helmet_global[f"height_{view}_mean"] / helmet_global[f"width_{view}_mean"]

        for postfix in ["_1", "_2"]:
            column_renames = {c: f"{c}_{view}{postfix}" for c in feature_cols}
            column_renames["nfl_player_id"] = f"nfl_player_id{postfix}"
            df = pd.merge(
                df,
                helmet_.rename(columns=column_renames),
                on=["datetime_ngs", f"nfl_player_id{postfix}"],
                how="left"
            )
            

        df = pd.merge(
            df,
            helmet_global,
            on="datetime_ngs",
            how="left"
        )
    
    # del df["datetime_ngs"] 残しといてもらえると嬉しい…。CNN終了後に消します。
    # del df["datetime"] # 0219 merge helmet smoothに使用するためキープ
    
    df = add_bbox_move_features(df)
    
    return reduce_dtype(df)

# 0219 update. カルマンスムーズ後のデータマージ。
def merge_helmet_smooth(df, helmet, meta):
    start_times = meta[["game_play", "start_time"]].drop_duplicates()
    start_times["start_time"] = pd.to_datetime(start_times["start_time"])

    helmet = pd.merge(helmet,
                      start_times,
                      on="game_play",
                      how="left")
    
    fps=59.94
    helmet["datetime"] = helmet["start_time"] + pd.to_timedelta(helmet["frame"] * (1 / fps), unit="s")
    helmet["datetime"] = pd.to_datetime(helmet["datetime"], utc=True)
    helmet["datetime_ngs"] = pd.DatetimeIndex(helmet["datetime"] + pd.to_timedelta(50, "ms")).floor("100ms").values
    helmet["datetime_ngs"] = pd.to_datetime(helmet["datetime_ngs"], utc=True)
    df["datetime_ngs"] = pd.to_datetime(df["datetime"], utc=True)
    
    feature_cols = ["bbox_smooth_velx","bbox_smooth_vely",
                    "bbox_smooth_x","bbox_smooth_y",
                    "bbox_smooth_outlier",
                    "bbox_smooth_accx","bbox_smooth_accy"]
    
    helmet_agg = helmet.groupby(["datetime_ngs", "nfl_player_id", "view"]).agg({c: "mean" for c in feature_cols}).reset_index()

    for view in ["Sideline", "Endzone"]:
        helmet_ = helmet_agg[helmet_agg["view"]==view].drop("view", axis=1)
    
        for postfix in ["_1", "_2"]:
            column_renames = {c: f"{c}_{view}{postfix}" for c in feature_cols}
            column_renames["nfl_player_id"] = f"nfl_player_id{postfix}"
            df = pd.merge(
                df,
                helmet_.rename(columns=column_renames),
                on=["datetime_ngs", f"nfl_player_id{postfix}"],
                how="left"
            )

    try:
        del df["datetime"]
    except:
        pass
    return reduce_dtype(df)


def read_csv_with_cache(filename, cfg: Config, usecols=None):
    cache_filename = filename.split(".")[0] + ".f"
    cache_path = os.path.join(cfg.CACHE_DIR, cache_filename)
    if not os.path.exists(cache_path):
        df = pd.read_csv(os.path.join(cfg.INPUT, filename), usecols=usecols)
        df = reduce_dtype(cast_player_id(df))
        df.to_feather(cache_path)
    return pd.read_feather(cache_path)


In [ ]:
import math
from tqdm import tqdm

class KalmanFilter:
    def __init__(self, 
                 num_sensor=2, 
                 timestep=0.02, 
                 acc_band=1, # change of xx pix/s during timestep. (unit can be changed)
                 sensor_errors=[5, 5], # measurement error(pix)
                 initial_state=[0,0,0,0], # x pix, vx pix/s, y pix, vy pix/s
                 clip_dist=20,
                 outlier_dist=100,
                 
                 ):
        if len(sensor_errors)!=num_sensor:
            raise Exception("len(sensor_errors) must be same as the num_sensor")
        self.state_dim = 4
        self.num_sensor = num_sensor
        self.timestep = timestep
        self.acc_band = acc_band
        self.sensor_errors = sensor_errors
        self.initial_state = np.array(initial_state)#.reshape(2,1)
        self.base_clip_dist = clip_dist
        self.base_outlier_dist = outlier_dist
        self.initial_cov_ratio = 100
        self.current_cov_ratio = self.initial_cov_ratio
        self.initialize_matrix()
    
    def initialize_matrix(self):
        # Motion Model
        self.A = np.array([[1, self.timestep, 0, 0],# x, vx, y, vy
                           [0, 1, 0, 0],
                           [0, 0, 1, self.timestep],
                           [0, 0, 0, 1]])
        # Jacobian of Observation Model
        self.C = np.array([[1,0,0,0],
                            [0,0,1,0]]) # x and y
        self.base_Q = self.get_cov_mat_self() # Covariance Matrix for motion
        self.base_R = self.get_cov_mat_sensor() # Covariance Matrix for observation
        self.Q = self.base_Q.copy() * self.initial_cov_ratio
        self.R = self.base_R.copy() * self.initial_cov_ratio
        self.outlier_dist = self.base_outlier_dist * self.initial_cov_ratio
        self.clip_dist = self.base_clip_dist * self.initial_cov_ratio
        self.x = self.initial_state
        self.P = np.eye(self.state_dim)
    
    def get_cov_mat_sensor(self):
        R = np.diag([e**2 for e in self.sensor_errors])
        return R
        
    def get_cov_mat_self(self):
        #Q = np.diag([self.acc_band**2, self.acc_band**2])
        Q = np.diag([0, self.acc_band**2, 0, self.acc_band**2])
        return Q
    
    def update_cov(self):
        reduction_ratio = 2
        self.current_cov_ratio = self.current_cov_ratio / reduction_ratio
        if self.current_cov_ratio > 1:
            self.Q = self.Q / reduction_ratio
            self.R = self.R / reduction_ratio
            self.outlier_dist = self.outlier_dist / reduction_ratio
            self.clip_dist = self.clip_dist / reduction_ratio
        else:
            self.Q = self.base_Q
            self.R = self.base_R
            self.outlier_dist = self.base_outlier_dist
            self.clip_dist = self.base_clip_dist
    
    def prior_estimate(self):
        """
        prior prediction of state
        and covariant matrix of prior prediction
        """
        x_prior = np.matmul(self.A, self.x) # 現在のstateと運動モデルから次のstateを予測
        P_prior = np.matmul(self.A, np.matmul(self.P, self.A.T)) + self.Q # motionの共分散にここまでの状態量の共分散を上乗せ、事前予測の共分散を得る
        return x_prior, P_prior
    
    def update(self, x_prior, P_prior, sensor_vals):
        """
        update
        - kalman gain
        - state
        - cov matrix
        """
        S = np.matmul(self.C, np.matmul(P_prior, self.C.T)) + self.R
        K = np.matmul(np.matmul(P_prior, self.C.T), np.linalg.inv(S))
        x = x_prior + np.matmul(K, sensor_vals - np.matmul(self.C, x_prior))
        P = np.matmul((np.eye(self.state_dim) - np.matmul(K, self.C)), P_prior)
        self.update_cov()
        return x, P
    
    def clip_by_dist(self, pos_prior, pos):
        delta = pos - pos_prior
        dist = math.sqrt(delta[0]**2 + delta[1]**2)
        # dist = np.sqrt((delta**2).sum())
        is_outlier = dist > self.outlier_dist
        if dist > self.clip_dist:
            pos = delta * min(dist, self.clip_dist) / (dist + 1e-12) + pos_prior
        if is_outlier:
            self.outlier_dist = self.outlier_dist + self.base_clip_dist
            self.clip_dist = self.clip_dist + self.base_clip_dist
        else:
            self.outlier_dist = max(self.outlier_dist - self.clip_dist, self.base_outlier_dist)
            self.clip_dist = max(self.clip_dist - self.clip_dist, self.base_clip_dist)            
        return pos, is_outlier
        
    def __call__(self, sensor_vals):
        x_prior, P_prior = self.prior_estimate()
        if np.isnan(sensor_vals[0]):
            self.x, self.P = x_prior, P_prior
        else:
            sensor_vals_clip, is_outlier = self.clip_by_dist(x_prior[::2], sensor_vals)
            if is_outlier:
                self.x, self.P = x_prior, P_prior
            else:
                self.x, self.P = self.update(x_prior, P_prior, sensor_vals_clip)
        return self.x
    
    def smooth_step(self, p_prior, p, x_prior, x):
        C = np.matmul(p, self.A.T).dot(np.linalg.inv(p_prior))
        x_smooth = x + C.dot(self.x - x_prior)#.squeeze()
        p_smooth = p + C.dot(self.P - p_prior).dot(C.T)
        return x_smooth, p_smooth
    
    def smoother(self, sequence_sensor_vals):
        x_priors = [self.x] # initial_value
        p_priors = [self.P]
        xs = [self.x]
        ps = [self.P]
        is_outlier_predicted = [0]
        for sensor_vals in sequence_sensor_vals[:]:#[1:]
            x_prior, P_prior = self.prior_estimate()
            if np.isnan(sensor_vals[0]):
                self.x, self.P = x_prior, P_prior
                is_outlier_predicted += [1]
            else:
                sensor_vals_clip, is_outlier = self.clip_by_dist(x_prior[::2], sensor_vals)
                if is_outlier:
                    self.x, self.P = x_prior, P_prior
                    is_outlier_predicted += [1]
                else:
                    self.x, self.P = self.update(x_prior, P_prior, sensor_vals_clip)
                    is_outlier_predicted += [0]
            # self.x, self.P = self.update(x_prior, P_prior, sensor_vals)
            x_priors.append(x_prior)
            p_priors.append(P_prior)
            xs.append(self.x)
            ps.append(self.P)
        xs_smooth = [self.x]
        ps_smooth = [self.P]
        for i in reversed(range(len(sequence_sensor_vals))):#-1
            self.x, self.P = self.smooth_step(p_priors[i+1], ps[i], x_priors[i+1], xs[i])
            xs_smooth.append(self.x)
            ps_smooth.append(self.P)
        xs_smooth = xs_smooth[::-1]
        ps_smooth = ps_smooth[::-1]
        return xs_smooth[1:], is_outlier_predicted[1:]
    
def apply_smoother_split(df_single_player, 
                         num_split_threshold_no_detect=20, # outlierも線引きしたほうがいいかもな。 
                         draw_output=False):
    xy_sequence = df_single_player[["cx", "cy"]].values
    frame_no_detected = df_single_player["frame"].values
    frame_no_to_xy = {fn: xy for fn,xy in zip(frame_no_detected, xy_sequence)}
    frame_no_min = frame_no_detected.min()
    frame_no_max = frame_no_detected.max()
    frame_range_all = np.arange(frame_no_min, frame_no_max+1)
    xy_sequence_all = np.array([frame_no_to_xy[fn] if fn in frame_no_detected else [np.nan, np.nan] for fn in frame_range_all])
    split_frame_range = []
    split_xy_sequence = []
    first_time_detected = 0
    last_time_detected = 0
    unseen_duration = 0
    missed = False
    for idx, f_no in enumerate(frame_range_all):
        if f_no in frame_no_detected:
            last_time_detected = idx
            if missed:
                first_time_detected = idx
            missed = False
            unseen_duration = 0
        else:
            unseen_duration += 1
            if missed:
                continue
            if unseen_duration >= num_split_threshold_no_detect:
                missed = True
                split_xy_sequence += [xy_sequence_all[first_time_detected:last_time_detected+1]]
                split_frame_range += [frame_range_all[first_time_detected:last_time_detected+1]]
            continue
    split_xy_sequence += [xy_sequence_all[first_time_detected:]]
    split_frame_range += [frame_range_all[first_time_detected:]]
    split_smoothed_pos = []
    split_smoothed_vel = []
    split_outlier_predicted = []
    for xy_sequence, frame_range in zip(split_xy_sequence, split_frame_range):
        initial_xyz = xy_sequence[~np.isnan(xy_sequence[:,0])][:3].mean(axis=0)
        kf = KalmanFilter(num_sensor=2,
                         timestep=1/60, 
                         acc_band=25, # change of N pix/s during timestep
                         sensor_errors=[5, 5], # measurement error(pix)
                         initial_state=[initial_xyz[0],0,initial_xyz[1],0], # x pix, vx pix/s, y pix, vy pix/s
                         clip_dist=20,
                         outlier_dist=100,
                         )

        # kalman smoother
        kf.initialize_matrix()
        smoothed, outlier_predicted = kf.smoother(xy_sequence)
        split_smoothed_pos.append(np.array(smoothed)[:,[0,2]])
        split_smoothed_vel.append(np.array(smoothed)[:,[1,3]])
        split_outlier_predicted.append(np.array(outlier_predicted))
    
    smoothed_pos = np.concatenate(split_smoothed_pos, axis=0)
    smoothed_vel = np.concatenate(split_smoothed_vel, axis=0)
    outlier_predicted = np.concatenate(split_outlier_predicted, axis=0)
    frame_range = np.concatenate(split_frame_range, axis=0)
    xy_sequence = np.concatenate(split_xy_sequence, axis=0)
    
    if draw_output:
        #plt.figure(figsize=[5,15])
        figs, axes = plt.subplots(1, 2, figsize=[15,3.5])
        axes[0].scatter(xy_sequence[:,0], xy_sequence[:,1], c=np.arange(len(xy_sequence)), s=35, vmin=0, vmax=len(xy_sequence))
        axes[0].plot(xy_sequence[:,0], xy_sequence[:,1])
        axes[0].invert_yaxis()
        axes[0].grid()
        axes[0].set_title(f" no postprocess")

        axes[1].scatter(smoothed_pos[:,0], smoothed_pos[:,1], c=np.arange(len(smoothed_pos)), s=35, vmin=0, vmax=len(smoothed_pos))
        axes[1].plot(smoothed_pos[:,0], smoothed_pos[:,1])
        axes[1].invert_yaxis()
        axes[1].grid()
        axes[1].set_title(f" kalman smooth")
        plt.show()
    return smoothed_pos, smoothed_vel, outlier_predicted, frame_range

def run_smoother_on_helmets(phase="train", path=None, num_split_threshold_no_detect = 10):
    if path is not None:
        if os.path.exists(path):
            df_smoothed = pd.read_csv(path)
            if 'Unnamed: 0' in df_smoothed.columns:
                df_smoothed = df_smoothed.drop(columns=['Unnamed: 0'])
            return df_smoothed
    
    draw_output = False
    
    df_helmet = pd.read_csv(f"{cfg.INPUT}/{phase}_baseline_helmets.csv")
    df_helmet["cx"] = df_helmet["left"] + df_helmet["width"] / 2
    df_helmet["cy"] = df_helmet["top"] + df_helmet["height"] / 2
    
    df_smoothed = []
    print("start kalman smooth.")
    for game_play in tqdm(df_helmet["game_play"].unique()):
        #print("GamePlay: ", game_play)
        # print(f"\r GamePlay {game_play}",end="")
        for view in ["Sideline", "Endzone"]:
            df_gp = df_helmet.query("game_play == @game_play and view == @view")
            smoothed_positions = []
            df_smoothed_velocities = []
            df_smoothed_gp = []
            vel_column_names = []
            for p_idx, nfl_player_id in enumerate(df_gp["nfl_player_id"].unique()):
                df_single_player = df_gp[df_gp["nfl_player_id"]==nfl_player_id]
                smoothed_pos, smoothed_vel, outlier_predicted, frame_range = apply_smoother_split(df_single_player, num_split_threshold_no_detect=num_split_threshold_no_detect, draw_output = draw_output)

                df_vel_player = pd.DataFrame(smoothed_vel, columns=[f"bbox_smooth_velx",f"bbox_smooth_vely"], index=frame_range)
                df_vel_player[["bbox_smooth_x","bbox_smooth_y"]] = smoothed_pos
                df_vel_player["bbox_smooth_outlier"] = outlier_predicted
                df_vel_player.loc[frame_range[:-1], [f"bbox_smooth_accx",f"bbox_smooth_accy"]] = smoothed_vel[1:] - smoothed_vel[:-1]
                df_vel_player["nfl_player_id"] = nfl_player_id
                df_smoothed_gp += [df_vel_player]

            df_smoothed_gp = pd.concat(df_smoothed_gp, axis=0).reset_index().rename(columns={"index":"frame"})
            df_smoothed_gp["view"] = view
            df_smoothed_gp["game_play"] = game_play

            df_smoothed.append(df_smoothed_gp)
    df_smoothed = pd.concat(df_smoothed, axis=0).reset_index(drop=True)
    df_smoothed[["bbox_smooth_x","bbox_smooth_y"]] = df_smoothed[["bbox_smooth_x","bbox_smooth_y"]].astype(np.float32)
    df_smoothed[["bbox_smooth_velx","bbox_smooth_vely"]] = df_smoothed[["bbox_smooth_velx","bbox_smooth_vely"]].astype(np.float32)
    df_smoothed[["bbox_smooth_accx","bbox_smooth_accy"]] = df_smoothed[["bbox_smooth_accx","bbox_smooth_accy"]].astype(np.float32)

    if path is not None:
        df_smoothed.to_csv(path, index=False)
    return df_smoothed

### bbox features

In [ ]:
from typing import List
## 畑追加箇所

def create_bbox_features(df: pd.DataFrame) -> pd.DataFrame:
    for view in ["Sideline", "Endzone"]:
        df = add_bbox_center_distance(df, view)
        df = add_bbox_from_step0_feature(df, view)
        df = add_shift_feature(df, view)
        df = add_diff_features(df, view)
        df = add_agg_bbox_feature(df, view)
    return df


def add_agg_bbox_feature(df:pd.DataFrame, view:str) -> pd.DataFrame:
    for agg in ['min', 'mean', 'std']:
        df[f'bbox_center_{view}_distance_{agg}'] = df.groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])[f'bbox_center_{view}_distance'].transform(agg)
        df[f'bbox_center_y_{view}_1_{agg}'] = df.groupby(['game_play', 'nfl_player_id_1'])[f'bbox_center_y_{view}_1'].transform(agg)
        df[f'bbox_center_y_std_{view}_1_{agg}'] = df.groupby(['game_play', 'nfl_player_id_1'])[f'bbox_center_y_std_{view}_1'].transform(agg)
        
    df[f'bbox_center_{view}_distance_ratio'] = df[f'bbox_center_{view}_distance_min'] / df[f'bbox_center_{view}_distance'] 
    return df


def add_bbox_center_distance(df:pd.DataFrame, view:str) -> pd.DataFrame:
    '''
    二人のplayerの中心間距離
    '''
    df[f'bbox_center_{view}_distance'] = distance(df[f'bbox_center_x_{view}_1'], 
                                                  df[f'bbox_center_y_{view}_1'], 
                                                  df[f'bbox_center_x_{view}_2'],
                                                  df[f'bbox_center_y_{view}_2'])
    
    return df

def add_bbox_from_step0_feature(df:pd.DataFrame, view:str) -> pd.DataFrame:
    for postfix in ["1", "2"]:
        step0_columns = [f'bbox_center_x_{view}_{postfix}', 
                         f'bbox_center_y_{view}_{postfix}']
        df = add_bbox_step0(df, step0_columns, postfix)
        df = add_distance_step0(df, step0_columns, view, postfix)
    return df

def add_bbox_step0(df:pd.DataFrame, step0_columns:List[str], postfix:str) -> pd.DataFrame:
    merge_key_columns =["game_play", f"nfl_player_id_{postfix}"]
        
    use_columns = deepcopy(merge_key_columns)
    use_columns.extend(step0_columns)
        
    _df = df[df['step']==0].copy()
    _df = _df[use_columns].drop_duplicates()
        
    rename_columns = [i+'_start' for i in step0_columns]
        
    _df.rename(columns=dict(zip(step0_columns, rename_columns)), 
                         inplace=True)
    
    df = pd.merge(df, 
                  _df, 
                  how='left', 
                  on=merge_key_columns)
    return df
    
def add_distance_step0(df: pd.DataFrame, step0_columns:List[str], view, postfix) -> pd.DataFrame:
        
    for column in step0_columns:
        df['diff_step_0_' + column] = df[column] - df[column + '_start']
        df['diff_step_0_' + column] = df[column] - df[column + '_start']
            
    df[f'distance_from_step0_{view}_{postfix}'] = distance(df[step0_columns[0]], 
                                                           df[step0_columns[1]],
                                                           df[step0_columns[0] + '_start'],
                                                           df[step0_columns[1] + '_start'])
    
    # 邪魔なので削除する
    df.drop(columns=[i + '_start' for i in step0_columns], inplace=True)
        
    return df
    
def add_shift_feature(df: pd.DataFrame, view: str) -> pd.DataFrame:
    result = []

    shift_columns = [f'distance_from_step0_{view}_1',
                     f'distance_from_step0_{view}_2',
                     f'bbox_center_{view}_distance',
                     f'bbox_center_y_{view}_1',
                     f'bbox_center_y_std_{view}_1']
    for i in [-5, -3, -1, 1, 3, 5]:

        _tmp = df.groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])[shift_columns].shift(i).add_prefix(f'shift_{i}_')
        result.append(_tmp)
        
    result = pd.concat(result, axis=1)
    df = pd.concat([df, result], axis=1)
    return df

def add_diff_feature(df: pd.DataFrame, column_name: str, shift_amount:List[int]) -> pd.DataFrame:
    for shift in shift_amount:
        df[f'diff_{shift}_0_{column_name}'] = df[column_name] - df[f'shift_{shift}_{column_name}'] 
    return df

def add_diff_features(df: pd.DataFrame, view:str) -> pd.DataFrame:
    shift_columns = [f'distance_from_step0_{view}_1',
                     f'distance_from_step0_{view}_2',
                     f'bbox_center_{view}_distance',
                     f'bbox_center_y_{view}_1',
                     f'bbox_center_y_std_{view}_1']
        
    for column in shift_columns:
        df = add_diff_feature(df, column, [-5, -3, -1, 1, 3, 5])

    return df

In [ ]:
def expand_contact_id(df):
    """
    Splits out contact_id into seperate columns.
    """
    df["game_play"] = df["contact_id"].str[:12]
    df["step"] = df["contact_id"].str.split("_").str[-3].astype("int")
    df["nfl_player_id_1"] = df["contact_id"].str.split("_").str[-2]
    df["nfl_player_id_2"] = df["contact_id"].str.split("_").str[-1]
    return cast_player_id(df)


def expand_helmet(cfg, df, phase="train"):
    helmet_cols = [
        "game_play",
        "view",
        "nfl_player_id",
        "frame",
        "left",
        "width",
        "top",
        "height"
    ]
    helmet = read_csv_with_cache(f"{phase}_baseline_helmets.csv", cfg, usecols=helmet_cols)
    meta = read_csv_with_cache(f"{phase}_video_metadata.csv", cfg)
    df = merge_helmet(df, helmet, meta)
    return df

# 0219 update
# kalman smooth後のhelmet情報をマージ。このタイミングでsmoothing走る。
def expand_helmet_smooth(cfg, df, phase="train"):
    if phase == "train":
        helmet = run_smoother_on_helmets(phase="train", 
                                         path=os.path.join(cfg.KMAT_PATH, f"output/{phase}_baseline_helmet_smooth.csv"),
                                         num_split_threshold_no_detect = 10)
    else:
        helmet = run_smoother_on_helmets(phase="test", 
                                         path=None, 
                                         num_split_threshold_no_detect = 10)
    
    meta = read_csv_with_cache(f"{phase}_video_metadata.csv", cfg)
    df = merge_helmet_smooth(df, helmet, meta)
    return df

with timer("load file"):
    tracking_cols = [
        "game_play",
        "nfl_player_id",
        "datetime",
        "step",
        "team",
        "position",
        "x_position",
        "y_position",
        "speed",
        "distance",
        "direction",
        "orientation",
        "acceleration",
        "sa"
    ]
    train_cols = [
        "game_play",
        "step",
        "nfl_player_id_1",
        "nfl_player_id_2",
        "contact",
        "datetime"
    ]
    
    if cfg.IS_TRAIN:
        tr_tracking = read_csv_with_cache("train_player_tracking.csv", cfg, usecols=tracking_cols)
        train = read_csv_with_cache("train_labels.csv", cfg, usecols=train_cols)
        split_defs = pd.read_csv(f"{cfg.KMAT_PATH}/output/game_fold.csv")

        
    te_tracking = read_csv_with_cache("test_player_tracking.csv", cfg, usecols=tracking_cols)
    sub = read_csv_with_cache("sample_submission.csv", cfg)
    test = expand_contact_id(sub)
    test = pd.merge(test, te_tracking[["step", "game_play", "datetime"]].drop_duplicates(), on=["game_play", "step"], how="left")


with timer("assign helmet metadata"):
    if cfg.IS_TRAIN:
        train = expand_helmet(cfg, train, "train")
        train = expand_helmet_smooth(cfg, train, "train")
    test = expand_helmet(cfg, test, "test")
    test = expand_helmet_smooth(cfg, test, "test")
    gc.collect()


In [ ]:
def merge_cols(df, tr, use_cols):
    key_cols = ["nfl_player_id", "step", "game_play"]
    use_cols = [c for c in use_cols if c in tr.columns]
    
    dst = pd.merge(
        df,
        tr[key_cols+use_cols].rename(columns={c: c+"_1" for c in use_cols}),
        left_on=["nfl_player_id_1", "step", "game_play"],
        right_on=key_cols,
        how="left"
    ).drop("nfl_player_id", axis=1)
    dst = pd.merge(
        dst,
        tr[key_cols+use_cols].rename(columns={c: c+"_2" for c in use_cols}),
        left_on=["nfl_player_id_2", "step", "game_play"],
        right_on=key_cols,
        how="left"
    ).drop("nfl_player_id", axis=1)
    
    return dst

## basic features (nyanp)

In [ ]:
def angle_diff(s1, s2):
    diff = s1 - s2
    
    return np.abs((diff + 180) % 360 - 180)

def distance(x1, y1, x2, y2):
    return np.sqrt(np.square(x2 - x1) + np.square(y2 - y1))


def make_basic_features(df):
    df["dx"] = df["x_position_1"] - df["x_position_2"]
    # df["dy"] = df["y_position_1"] - df["y_position_2"]
    df["distance"] = distance(df["x_position_1"], df["y_position_1"], df["x_position_2"], df["y_position_2"])
    df["different_team"] = df["team_1"] != df["team_2"]
    df["anglediff_dir1_dir2"] = angle_diff(df["direction_1"], df["direction_2"])
    df["anglediff_ori1_ori2"] = angle_diff(df["orientation_1"], df["orientation_2"])
    df["anglediff_dir1_ori1"] = angle_diff(df["direction_1"], df["orientation_1"])
    df["anglediff_dir2_ori2"] = angle_diff(df["direction_2"], df["orientation_2"])
    return df

def tracking_prep(tracking):
    for c in ["direction", "orientation", "acceleration", "sa", "speed", "distance"]:
        tracking[f"{c}_p1"] = tracking.groupby(["nfl_player_id", "game_play"])[c].shift(-1)
        tracking[f"{c}_m1"] = tracking.groupby(["nfl_player_id", "game_play"])[c].shift(1)
        tracking[f"{c}_p1_diff"] = tracking[c] - tracking[f"{c}_p1"]
        tracking[f"{c}_m1_diff"] = tracking[f"{c}_m1"] - tracking[c]
        
        if c in ["direction", "orientation"]:
            tracking[f"{c}_p1_diff"] = np.abs((tracking[f"{c}_p1_diff"] + 180) % 360 - 180)
            tracking[f"{c}_m1_diff"] = np.abs((tracking[f"{c}_m1_diff"] + 180) % 360 - 180)
        
    return tracking


def tracking_agg_features(df, tracking):
    """トラッキングデータのstep内での単純集計。他プレイヤーの動きを見る"""
    
    team_agg = tracking.groupby(["game_play", "step", "team"]).agg(
        x_position_team_mean = pd.NamedAgg("x_position", "mean"),
        y_position_team_mean = pd.NamedAgg("y_position", "mean"),
        speed_team_mean = pd.NamedAgg("speed", "mean"),
        acceleration_team_mean = pd.NamedAgg("acceleration", "mean"),
        sa_team_mean = pd.NamedAgg("sa", "mean")
    )
    agg = tracking.groupby(["game_play", "step"]).agg(
        x_position_mean = pd.NamedAgg("x_position", "mean"),
        y_position_mean = pd.NamedAgg("y_position", "mean"),
        speed_mean = pd.NamedAgg("speed", "mean"),
        acceleration_mean = pd.NamedAgg("acceleration", "mean"),
        sa_mean = pd.NamedAgg("sa", "mean")
    )
    player_agg = tracking[tracking["step"]>=0].groupby(["game_play", "nfl_player_id"]).agg(
        sa_player_mean = pd.NamedAgg("sa", "mean"),
        sa_player_max = pd.NamedAgg("sa", "max"),
        acceleration_player_mean = pd.NamedAgg("acceleration", "mean"),
        acceleration_player_max = pd.NamedAgg("acceleration", "max"),
        speed_player_mean = pd.NamedAgg("speed", "mean"),
        speed_player_max = pd.NamedAgg("speed", "max"),
    )

    for postfix in ["_1", "_2"]:
        df = pd.merge(
            df,
            team_agg.rename(columns={c: c+postfix for c in team_agg.columns}).reset_index(),
            left_on=["game_play", "step", f"team{postfix}"],
            right_on=["game_play", "step", "team"],
            how="left"
        ).drop("team", axis=1)
        
        player_agg_renames = {c: c+postfix for c in player_agg.columns}
        player_agg_renames["nfl_player_id"] = f"nfl_player_id{postfix}"

        df = pd.merge(
            df,
            player_agg.reset_index().rename(columns=player_agg_renames),
            on=["game_play", f"nfl_player_id{postfix}"],
            how="left"
        )

    
    df = pd.merge(df, agg, on=["game_play", "step"], how="left")
    return df


In [ ]:
def second_nearest_distance(df, tr_tracking, target="1"):
    stacked = pd.concat([
        df[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "distance", "different_team"]],
        df[["game_play", "step", "nfl_player_id_2", "nfl_player_id_1", "distance", "different_team"]].rename(columns={"nfl_player_id_2": "nfl_player_id_1", "nfl_player_id_1": "nfl_player_id_2"}),
    ])
    
    def _build(df, s, postfix=""):
        s["distance_rank"] = s.groupby(["game_play", "step", "nfl_player_id_1"])["distance"].rank()

        stacked_1st = s[s["distance_rank"]==1].drop(["nfl_player_id_2", "distance_rank", "different_team"], axis=1)
        stacked_1st.columns = ["game_play", "step", f"nfl_player_id_{target}", f"distance_1st_{target}{postfix}"]
        stacked_2nd = s[s["distance_rank"]==2].drop(["nfl_player_id_2", "distance_rank", "different_team"], axis=1)
        stacked_2nd.columns = ["game_play", "step", f"nfl_player_id_{target}", f"distance_2nd_{target}{postfix}"]

        stacked_mrg = pd.merge(stacked_1st, stacked_2nd, on=["game_play", "step", f"nfl_player_id_{target}"], how="left")
        stacked_mrg[f"distance_diff_2nd_to_1st_{target}{postfix}"] = stacked_mrg[f"distance_2nd_{target}{postfix}"] - stacked_mrg[f"distance_1st_{target}{postfix}"]

        df = pd.merge(df, stacked_mrg, on=["game_play", "step", f"nfl_player_id_{target}"], how="left")
        return df
    
    df = _build(df, stacked)
    df = _build(df, stacked[stacked["different_team"]], "_different_team")
    return df
    


def bbox_distance_around_player(df):
    """距離など、pairwiseに計算された特徴量を集計しなおす
    対象のプレイヤーの周りに他プレイヤーが密集しているか？
    """

    stacked = pd.concat([
        df[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "bbox_center_Sideline_distance", "bbox_center_Endzone_distance"]],
        df[["game_play", "step", "nfl_player_id_2", "nfl_player_id_1", "bbox_center_Sideline_distance", "bbox_center_Endzone_distance"]].rename(columns={"nfl_player_id_2": "nfl_player_id_1", "nfl_player_id_1": "nfl_player_id_2"}),
    ])
    def _arg_min(s):
        try:
            return np.nanargmin(s.values)
        except ValueError:
            return np.nan
    feature_cols = [
        "mean_bbox_distance_around_player"
        "min_bbox_distance_around_player",
        "idxmin_bbox_distance_around_player"
    ]
    aggfunc = ["mean", "min", _arg_min]
    
    def _merge_stacked_df(df, s, postfix=""):
        s = s.groupby(["game_play", "step", "nfl_player_id_1"]).agg({
            "bbox_center_Sideline_distance": aggfunc,
            "bbox_center_Endzone_distance": aggfunc,
        }).reset_index()
        s = reduce_dtype(s)
        columns = ["nfl_player_id"] + [f"{f}_Sideline{postfix}" for f in feature_cols] + [f"{f}_Endzone{postfix}" for f in feature_cols]
        s.columns = ["game_play", "step"] + columns

        df = pd.merge(
            df, 
            s.rename(columns={c: f"{c}_1" for c in columns}),
            on=["game_play", "step", "nfl_player_id_1"],
            how="left"
        )
        df = pd.merge(
            df, 
            s.rename(columns={c: f"{c}_2" for c in columns}),
            on=["game_play", "step", "nfl_player_id_2"],
            how="left"
        )
        return df
    
    def _merge_stacked_df_pairwise(df, s):
        s = s.groupby(["game_play", "nfl_player_id_1", "nfl_player_id_2"]).agg({
            "bbox_center_Sideline_distance": aggfunc,
            "bbox_center_Endzone_distance": aggfunc,
        }).reset_index()
        s = reduce_dtype(s)
        columns = [f"{f}_Sideline_pair" for f in feature_cols] + [f"{f}_Endzone_pair" for f in feature_cols]
        s.columns = ["game_play", "nfl_player_id_1", "nfl_player_id_2"] + columns
        df = pd.merge(
            df, 
            s,
            on=["game_play", "nfl_player_id_1", "nfl_player_id_2"],
            how="left"
        )
        return df
    
    df = _merge_stacked_df(df, stacked, "")
    df = _merge_stacked_df_pairwise(df, stacked)

    return df

def distance_around_player(df, on_full_sample = False):
    """距離など、pairwiseに計算された特徴量を集計しなおす
    対象のプレイヤーの周りに他プレイヤーが密集しているか？
    """

    stacked = pd.concat([
        df[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "distance", "different_team"]],
        df[["game_play", "step", "nfl_player_id_2", "nfl_player_id_1", "distance", "different_team"]].rename(columns={"nfl_player_id_2": "nfl_player_id_1", "nfl_player_id_1": "nfl_player_id_2"}),
    ])

    def _arg_min(s):
        try:
            return np.nanargmin(s.values)
        except ValueError:
            return np.nan
    
    if on_full_sample:
        # minはハードサンプルだけで見たときと同じなので飛ばす
        feature_cols = [
            "mean_distance_around_player_full",
            "std_distance_around_player_full",
            "idxmin_distance_aronud_player_full"
        ]
        aggfunc = ["mean", "std", _arg_min]
    else:
        feature_cols = [
            "mean_distance_around_player",
            "min_distance_around_player",
            "std_distance_around_player",
            # "idxmin_distance_aronud_player"
        ]
        aggfunc = ["mean", "min", "std"] #, _arg_min]


    def _merge_stacked_df(df, s, postfix=""):
        s = s.groupby(["game_play", "step", "nfl_player_id_1"]).agg({"distance": aggfunc}).reset_index()
        s = reduce_dtype(s)
        columns = ["nfl_player_id"] + [f"{f}{postfix}" for f in feature_cols]
        s.columns = ["game_play", "step"] + columns

        df = pd.merge(
            df, 
            s.rename(columns={c: f"{c}_1" for c in columns}),
            on=["game_play", "step", "nfl_player_id_1"],
            how="left"
        )
        df = pd.merge(
            df, 
            s.rename(columns={c: f"{c}_2" for c in columns}),
            on=["game_play", "step", "nfl_player_id_2"],
            how="left"
        )
        return df
    
    def _merge_stacked_df_pairwise(df, s):
        s = s.groupby(["game_play", "nfl_player_id_1", "nfl_player_id_2"]).agg({"distance": aggfunc}).reset_index()
        s = reduce_dtype(s)
        columns = [f"{f}_pair" for f in feature_cols]
        s.columns = ["game_play", "nfl_player_id_1", "nfl_player_id_2"] + columns
        df = pd.merge(
            df, 
            s,
            on=["game_play", "nfl_player_id_1", "nfl_player_id_2"],
            how="left"
        )
        return df
        
    df = _merge_stacked_df(df, stacked, "")
    df = _merge_stacked_df(df, stacked[stacked["different_team"]], "_different_team")
    df = _merge_stacked_df_pairwise(df, stacked)

    if on_full_sample:
        df["step_diff_to_min_distance_full"] = df["step"] - df["idxmin_distance_aronud_player_full_pair"]
    else:
        pass
        # df["step_diff_to_min_distance"] = df["step"] - df["idxmin_distance_aronud_player_pair"]
    return df


def step_feature(df, tracking):
    """stepの割合など
    １プレーの長さや、その中で前半後半のどの辺のステップなのかに意味がある
    （例えば、プレー開始直後に地面とコンタクトする可能性は低い）
    """
    
    # 予測対象フレーム
    df["step_max_1"] = df.groupby("game_play")["step"].transform("max")
    df["step_ratio_1"] = (df["step"] / df["step_max_1"]).astype(np.float32)

    # 全体
    #step_agg = tracking.groupby("game_play")["step"].agg(["min", "max"])
    #step_agg.columns = ["step_min_2", "step_max_2"]
    #df = pd.merge(df, step_agg, left_on="game_play", right_index=True, how="left")
    #df["step_ratio_2"] = df["step"] / df["step_max_2"]
    return df

def aspect_ratio_feature(df, drop_original=False):
    for postfix in ["_1", "_2"]:
        for view in ["Sideline", "Endzone"]:
            df[f"aspect_{view}{postfix}"] = df[f"height_{view}{postfix}"] / df[f"width_{view}{postfix}"]
            
            if drop_original:
                del df[f"height_{view}{postfix}"]
                del df[f"width_{view}{postfix}"]   
    return df

def misc_features_after_agg(df):
    """集約した特徴量とJoinした特徴量の比率や差など、他の特徴量を素材に作る特徴量"""
    df["distance_from_mean_1"] = distance(
        df["x_position_1"],
        df["y_position_1"],
        df["x_position_mean"],
        df["y_position_mean"],
    )
    df["distance_from_mean_2"] = distance(
        df["x_position_2"],
        df["y_position_2"],
        df["x_position_mean"],
        df["y_position_mean"],
    )
    df["distance_team2team"] = distance(
        df["x_position_team_mean_1"],
        df["y_position_team_mean_1"],
        df["x_position_team_mean_2"],
        df["y_position_team_mean_2"],
    )
    df["speed_diff_1_2"] = df["speed_1"] - df["speed_2"]
    df["speed_diff_1_team"] = df["speed_1"] - df["speed_team_mean_1"]
    df["speed_diff_2_team"] = df["speed_2"] - df["speed_team_mean_2"]
    df["distance_mean_in_play"] = df.groupby("game_play")["distance"].transform("mean")
    #df["distance_std_in_play"] = df.groupby("game_play")["distance"].transform("std")
    #df["distance_team2team_mean_in_play"] = df.groupby("game_play")["distance_team2team"].transform("mean")
    
    # player pairが一番近づいた瞬間の距離と現在の距離の比
    df["distance_ratio_distance_to_min_pair_distance"] = df["distance"] / df["min_distance_around_player_pair"]
    
    # player pairの距離と、現在一番player1の近くにいるplayerとの距離比
    df["distance_ratio_distance_to_min_distance_around_player_1"] = df["distance"] / df["min_distance_around_player_1"]
    df["distance_ratio_distance_to_min_distance_around_player_2"] = df["distance"] / df["min_distance_around_player_2"]
    df["distance_ratio_distance_to_min_distance_around_player_diffteam_1"] = df["distance"] / df["min_distance_around_player_different_team_1"]
    df["distance_ratio_distance_to_min_distance_around_player_diffteam_2"] = df["distance"] / df["min_distance_around_player_different_team_2"]

    #df["distance_ratio_distance_1"] = df["distance_1"] / df["distance"]
    #df["distance_ratio_distance_2"] = df["distance_2"] / df["distance"]
   
    # 進行方向以外の加速度成分
    #df["sa_ratio_1"] = np.abs(df["sa_1"] / df["acceleration_1"])
    #df["sa_ratio_2"] = np.abs(df["sa_2"] / df["acceleration_2"])
    return df


In [ ]:
def t0_feature(df, tracking):
    # step=0時点での統計量を使った特徴量
    on_play_start = tracking[tracking["step"]==0].reset_index(drop=True)
    on_play_start.rename(columns={"x_position": "x_position_start", "y_position": "y_position_start"}, inplace=True)

    # step=0時点でのx位置は？
    mean_x_on_play = on_play_start.groupby("game_play")["x_position_start"].mean()
    mean_x_on_play.name = "x_position_mean_on_start"

    feature_cols = ["nfl_player_id", "x_position_start", "y_position_start"]
    df = pd.merge(
        df, 
        on_play_start[["game_play"]+feature_cols].rename(columns={c: f"{c}_1" for c in feature_cols}),
        on=["game_play", "nfl_player_id_1"],
        how="left"
    )
    df = pd.merge(
        df, 
        on_play_start[["game_play"]+feature_cols].rename(columns={c: f"{c}_2" for c in feature_cols}),
        on=["game_play", "nfl_player_id_2"],
        how="left"
    )
    df = pd.merge(
        df,
        mean_x_on_play.reset_index(),
        on="game_play",
        how="left"
    )
    # step=0時点からどれくら移動しているか？
    df["distance_from_start_1"] = distance(
        df["x_position_start_1"],
        df["y_position_start_1"],
        df["x_position_1"],
        df["y_position_1"]
    )
    df["distance_from_start_2"] = distance(
        df["x_position_start_2"],
        df["y_position_start_2"],
        df["x_position_2"],
        df["y_position_2"]
    )
    return df

def shift_of_player(df, tracking, shifts, add_diff=False, player_id="1"):
    step_orig = tracking["step"].copy()
    feature_cols = ["x_position", "y_position", 
                    "speed", "orientation", "direction", 
                    "acceleration", "distance", "sa"]

    for shift in shifts:
        tracking["step"] = step_orig - shift
        f_or_p = "future" if shift > 0 else "past"
        abs_shift = np.abs(shift)
        
        renames = {c: f"{c}_{f_or_p}{abs_shift}_{player_id}" for c in feature_cols}
        renames["nfl_player_id"] = f"nfl_player_id_{player_id}"
        
        df = pd.merge(
            df, 
            tracking[["step", "game_play", "nfl_player_id"]+feature_cols].rename(columns=renames),
            on=["step", "game_play", f"nfl_player_id_{player_id}"],
            how="left"
        )
        if add_diff:
            for c in feature_cols:
                if c in ["orientation", "direction"]:
                    df[f"{c}_diff_{shift}_{player_id}"] = angle_diff(df[f"{c}_{player_id}"], df[renames[c]])
                else:
                    df[f"{c}_diff_{shift}_{player_id}"] = df[f"{c}_{player_id}"] - df[renames[c]]
        
    tracking["step"] = step_orig
    
    return df

def bbox_y_endzone_diff_feature(df, distance_th=3.0):
    """近傍の選手のbboxのy座標との差をとる（転んでいる選手を検出）"""
    bbox_y_neighbor = df[[
        "game_play", "step", "nfl_player_id_1", "nfl_player_id_2",
        "bbox_center_y_Endzone_1", "bbox_center_y_Endzone_2", "distance"
    ]].copy()
    
    if df["distance"].max() > distance_th:
        bbox_y_neighbor = bbox_y_neighbor[df["distance"] < distance_th]

    bbox_y_neighbor["weight"] = 1 / (bbox_y_neighbor["distance"] + 0.1)

    bbox_y_neighbor_swap = bbox_y_neighbor.copy()
    bbox_y_neighbor_swap.columns = [
        "game_play", "step", "nfl_player_id_2", "nfl_player_id_1",
        "bbox_center_y_Endzone_2", "bbox_center_y_Endzone_1", "distance", "weight"
    ]
    bbox_y_neighbor = pd.concat([bbox_y_neighbor, bbox_y_neighbor_swap[bbox_y_neighbor.columns]])
    
    del bbox_y_neighbor_swap

    # player_id_1の近傍distance_th以内にいる他プレーヤーのbbox座標を集約する
    
    bbox_y_neighbor["wy"] = bbox_y_neighbor["weight"] * bbox_y_neighbor["bbox_center_y_Endzone_2"]

    bbox_neighbor_agg = bbox_y_neighbor.groupby(["game_play", "step", "nfl_player_id_1"]).agg(
        neighbor_count_1 = pd.NamedAgg("nfl_player_id_2", "count"),
        neighbor_y_wy_sum_1 = pd.NamedAgg("wy", "sum"),
        neighbor_y_w_sum_1 =pd.NamedAgg("weight", "sum"),
        neighbor_y_mean_1 = pd.NamedAgg("bbox_center_y_Endzone_2", "mean"),
        #neighbor_y_count = pd.NamedAgg("bbox_center_y_Endzone_2", "count"),
    ).reset_index()

    bbox_neighbor_agg["neighbor_y_w_mean_1"] = bbox_neighbor_agg["neighbor_y_wy_sum_1"] / bbox_neighbor_agg["neighbor_y_w_sum_1"]

    del bbox_neighbor_agg["neighbor_y_wy_sum_1"]
    del bbox_neighbor_agg["neighbor_y_w_sum_1"]

    # 転んでいる人を拾いたいので、player_id_1だけ特徴量を作る。id_2側に追加してもスコアは上がらない。
    df = pd.merge(df, bbox_neighbor_agg, on=["game_play", "step", "nfl_player_id_1"], how="left")
    df["bbox_y_endzone_diff_from_neighbors_1"] = df["bbox_center_y_Endzone_1"] - df["neighbor_y_mean_1"]
    df["bbox_y_endzone_diff_from_weighted_neighbors_1"] = df["bbox_center_y_Endzone_1"] - df["neighbor_y_w_mean_1"]
    
    del df["neighbor_y_mean_1"]
    del df["neighbor_y_w_mean_1"]
    return df

## CNN系特徴(kmat)


In [ ]:
"""
ロード及び前処理まわり。
"""
def cv2bgr_to_tf32(img):
    #return tf.cast(img[:,:,::-1], tf.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return tf.cast(img, tf.float32)

def make_rectangle(df):
    top = df.top.values.reshape(-1)
    left = df.left.values.reshape(-1)
    width = df.width.values.reshape(-1)
    height = df.height.values.reshape(-1)
    bottom = top + height
    right = left + width
    top = top.tolist() 
    left = left.tolist()
    bottom = bottom.tolist()
    right = right.tolist()
    rectangles=[]
    for i in range(len(top)):
        rectangle = [[left[i], top[i]],[right[i], top[i]],
                     [right[i], bottom[i]],[left[i], bottom[i]]]
        rectangles.append(rectangle)
    return rectangles

def load_helmet_meta(cfg, phase="train"):
    helmet = pd.read_csv(f"{cfg.INPUT}/{phase}_baseline_helmets.csv")
    meta = pd.read_csv(f"{cfg.INPUT}/{phase}_video_metadata.csv", parse_dates=["start_time", "end_time", "snap_time"])

    #helmet = read_csv_with_cache(f"{phase}_baseline_helmets.csv", cfg)
    #meta = read_csv_with_cache(f"{phase}_video_metadata.csv", cfg)
    #meta["start_time"] = pd.to_datetime(meta["start_time"], utc=True)
    return helmet, meta

def prepare_matching_dataframe(game_play, tr_tracking, helmets, meta, view="Sideline", fps=59.94, only_center_of_step=True):
    tr_tracking = tr_tracking.query("game_play == @game_play").copy()
    gp_helms = helmets.query("game_play == @game_play").copy()

    start_time = meta.query("game_play == @game_play and view == @view")[
        "start_time"
    ].values[0]

    gp_helms["datetime"] = (
        pd.to_timedelta(gp_helms["frame"] * (1 / fps), unit="s") + start_time
    )
    gp_helms["datetime"] = pd.to_datetime(gp_helms["datetime"], utc=True)
    gp_helms["datetime_ngs"] = (
        pd.DatetimeIndex(gp_helms["datetime"] + pd.to_timedelta(50, "ms"))
        .floor("100ms")
        .values
    )
    gp_helms["datetime_ngs"] = pd.to_datetime(gp_helms["datetime_ngs"], utc=True)
    gp_helms["delta_from_round_val"] = (gp_helms["datetime"]-gp_helms["datetime_ngs"]).dt.total_seconds()
    
    gp_helms["center_frame_of_step"] = np.abs(gp_helms["delta_from_round_val"]) 
    gp_helms["center_frame_of_step"] = gp_helms["center_frame_of_step"].values==gp_helms.groupby("datetime_ngs")["center_frame_of_step"].transform("min").values
    if only_center_of_step:
        gp_helms = gp_helms[gp_helms["center_frame_of_step"]].drop(columns=["center_frame_of_step"])
    gp_helms = gp_helms[gp_helms["view"]==view]

    tr_tracking["datetime_ngs"] = pd.to_datetime(tr_tracking["datetime"], utc=True)
    gp_helms = gp_helms.merge(
        tr_tracking[["datetime_ngs","step","x_position","y_position", "nfl_player_id"]],
        left_on=["datetime_ngs", "nfl_player_id"],
        right_on=["datetime_ngs", "nfl_player_id"],
        how="right", # left -> outer to find the player out of the frame
    )
    gp_helms["view"] = gp_helms["view"].fillna(view)
    gp_helms["frame"] = gp_helms.groupby(["datetime_ngs"])["frame"].transform("mean")
    gp_helms["game_play"] = game_play
    
    gp_helms = gp_helms[~gp_helms["step"].isna()]
    gp_helms = gp_helms[~gp_helms["frame"].isna()]
    # 複数minimumが存在するケースもあるので気を付ける(あとでグループ平均とるなど)
    return gp_helms
    

def prepare_cnn_dataframe(game_play, labels, helmets, meta, view="Sideline", fps=59.94):
    gp_labs = labels.query("game_play == @game_play").copy()
    gp_helms = helmets.query("game_play == @game_play").copy()

    start_time = meta.query("game_play == @game_play and view == @view")[
        "start_time"
    ].values[0]

    gp_helms["datetime"] = (
        pd.to_timedelta(gp_helms["frame"] * (1 / fps), unit="s") + start_time
    )
    gp_helms["datetime"] = pd.to_datetime(gp_helms["datetime"], utc=True)
    gp_helms["datetime_ngs"] = (
        pd.DatetimeIndex(gp_helms["datetime"] + pd.to_timedelta(50, "ms"))
        .floor("100ms")
        .values
    )
    gp_helms["datetime_ngs"] = pd.to_datetime(gp_helms["datetime_ngs"], utc=True)
    gp_helms["delta_from_round_val"] = (gp_helms["datetime"]-gp_helms["datetime_ngs"]).dt.total_seconds()
    
    if "datetime_ngs" not in gp_labs.columns:
        gp_labs["datetime_ngs"] = pd.to_datetime(gp_labs["datetime"], utc=True)
    gp_helms = gp_helms.merge(
        gp_labs[["datetime_ngs","step"]].drop_duplicates(),
        on=["datetime_ngs"],
        how="left",
    )
    gp_helms["center_frame_of_step"] = np.abs(gp_helms["delta_from_round_val"]) 
    gp_helms["center_frame_of_step"] = gp_helms["center_frame_of_step"].values==gp_helms.groupby("step")["center_frame_of_step"].transform("min").values
    # 複数minimumが存在するケースもあるので気を付ける(あとでグループ平均とるなど)
    
    ###追加した。ok?
    gp_helms = gp_helms[gp_helms["view"]==view]
    
    return gp_labs, gp_helms
    

    
"""
- Camaro-san Reader
"""
# import the necessary packages
from threading import Thread

if sys.version_info >= (3, 0):
    from queue import Queue
else:
    from Queue import Queue


class FileVideoStream:
    def __init__(self, cv2_stream, transform=None, queue_size=16):
        self.stream = cv2_stream#cv2.VideoCapture(path)
        self.stopped = False
        self.transform = transform

        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queue_size)
        # intialize thread
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True

    def start(self):
        # start a thread to read frames from the file video stream
        self.thread.start()
        return self

    def update(self):
        # keep looping infinitely
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                break

            if not self.Q.full():
                (grabbed, frame) = self.stream.read()

                if not grabbed:
                    self.stopped = True
                
                if grabbed and self.transform:
                    frame = self.transform(frame)
                    # add the frame to the queue
                    self.Q.put(frame)
            else:
                time.sleep(0.01)  # Rest for 10ms, we have a full queue

        self.stream.release()

    def read(self):
        # return next frame in the queue
        return self.Q.get()

    def running(self):
        return self.more() or not self.stopped

    def more(self):
        # return True if there are still frames in the queue. If stream is not stopped, try to wait a moment
        tries = 0
        while self.Q.qsize() == 0 and not self.stopped and tries < 5:
            time.sleep(0.1)
            tries += 1

        return self.Q.qsize() > 0

    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True
        # wait until stream resources are released (producer thread might be still grabbing frame)
        self.thread.join()
        
"""
video loader for cnn
"""
        
class Video2Input():
    def __init__(self,
                 game_play_view,
                    video_path,
                    labels,
                 helms,
                frame_interval=1,
                only_center_frame_of_step=True):
        
        VIDEO_CODEC = "MP4V"
        self.video_name = os.path.basename(video_path)
        print(f"Preparing {self.video_name}")
        self.labels = labels.copy()#.query("video == @self.video_name").copy()
        self.helms = helms.query("video == @self.video_name").copy()
        min_frame = self.helms["frame"].values[np.argmin(self.helms["step"].values)]
        self.start_frame = max(0, min_frame-5)
        self.vidcap = cv2.VideoCapture(video_path)
        self.fps = self.vidcap.get(cv2.CAP_PROP_FPS)
        #self.width = int(self.vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
        #self.height = int(self.vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.total_frames = int(self.vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.current_frame = self.start_frame
        self.vidcap.set(cv2.CAP_PROP_POS_FRAMES, self.start_frame)
        self.interval = frame_interval
        
            
        
        self.vidcap = FileVideoStream(self.vidcap, 
                                      #transform=cv2bgr_to_tf32,
                                     transform= lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
                                     ).start()
    
    def get_next(self, only_center_frame_of_step=True):
        """
        only_center_frame_of_step:
            if True, use only center_frame_of_step and neglect other frames.
        """
        while True:
            #it_worked, img = self.vidcap.read()
            img = self.vidcap.read()
            #if not it_worked:
            if not self.vidcap.running():
                self.vidcap.stop()
                return self.current_frame, None
            # We need to add 1 to the frame count to match the label frame index
            # that starts at 1
            self.current_frame += 1        
            if self.current_frame % self.interval != 0:
                continue

            # Now, add the boxes
            df_frame_helms =  self.helms[self.helms["frame"]==self.current_frame]#uery("frame == @self.current_frame")
            if len(df_frame_helms)==0:
                continue
            step_no = df_frame_helms["step"].iloc[0]
            if only_center_frame_of_step:
                if df_frame_helms["center_frame_of_step"].iloc[0]==False:
                    continue
            df_frame_label =  self.labels[self.labels["step"]==step_no]
            if len(df_frame_label)==0:
                continue

            rectangles = make_rectangle(df_frame_helms)
            player_id = df_frame_helms["nfl_player_id"].values.astype(int).tolist()
            player_id_1 = df_frame_label["nfl_player_id_1"].values.astype(int)#.tolist()
            player_id_2 = df_frame_label["nfl_player_id_2"].values.astype(int)#.tolist()
            
            # remove players not in the image
            set_player = set([0]+player_id)
            player_1_exist = [p in set_player for p in player_id_1]
            player_2_exist = [p in set_player for p in player_id_2]
            players_exist = np.logical_and(player_1_exist, player_2_exist)
            
            player_id_1 = player_id_1[players_exist]
            player_id_2 = player_id_2[players_exist]
            contact_labels = df_frame_label["contact"].values[players_exist]#.tolist()
            num_contact_labels = len(contact_labels)
            num_player = len(set_player) - 1

            contact_pairlabels = np.vstack([player_id_1, player_id_2, contact_labels]).T
            
            if num_player==0:
                continue
            
            data = {"rgb": tf.cast(img, tf.float32),#img,#cv2bgr_to_tf32(img), 
                    "rectangles": tf.cast(rectangles, tf.float32),
                    "player_id": tf.cast(player_id, tf.int32),
                    "contact_pairlabels": tf.cast(contact_pairlabels, tf.int32),
                    "num_labels": tf.cast(num_contact_labels, tf.int32),
                    "num_player": num_player,
                    "img_height": 720,
                    "img_width": 1280,
                    }
            step_frame_no = [step_no, self.current_frame] # single step(100ms) contains multiple frames
            return step_frame_no, data   


class DataStacker:
    def __init__(self, batch_size=2, as_list=False):
        self.stacked = None
        self.length = 0
        self.batch_size = batch_size
        self.as_list = as_list
    
    def add(self, data):
        if self.stacked is None:
            self.stacked = {k: [data[k]] for k in data.keys()}
        else:
            self.stacked = {k: self.stacked[k]+[data[k]] for k in data.keys()}
        self.length += 1
        if self.length > self.batch_size:
            raise Exception("stacked too much")
        
    def get_if_ready(self, reset=True, neglect_readiness=False):
        if self.length == self.batch_size or neglect_readiness:
            is_ready = True
            if self.as_list:
                stacked = self.stacked
            else:
                stacked = {k: tf.stack(self.stacked[k]) for k in self.stacked.keys()}
            if reset:
                self.reset()
        else:
            stacked = None
            is_ready = False
        return is_ready, stacked
    
    def is_ready(self):
        return self.length == self.batch_size
        
    def reset(self):
        self.stacked = None
        self.length = 0        
        

"""
CNN main実行系関数。
foldとるところ少し一時的。。。
"""

def build_model(input_shape, output_shape, load_path, num_max_pair=153, skip_build_map_model=False):
    from train_utils.tf_Augmentations_detection import ComposeNopos, Center_Crop
    
    is_crop_first_model = True if "cropfirst" in load_path else False
    print(f"load cnn weights {load_path}. crop model = {is_crop_first_model}")
    
    model_params = {"input_shape": input_shape,
                    "output_shape": output_shape,  
                    "weight_file": load_path,
                    "is_train_model": False,
                    "crop_first_model": is_crop_first_model,
                   }
    model = NFLContact(**model_params)
    if skip_build_map_model: # when ensemble, not need to build it again
        model.use_map_model = True
    else:
        model.combine_map_model(f"{cfg.KMAT_PATH}/model/weights/map_model_final_weights.h5")
    transforms = [
                  Center_Crop(p=1, min_height=input_shape[0], min_width=input_shape[1]),
                  ]
    transforms = ComposeNopos(transforms)
    preprocessor = lambda x: inference_preprocess(x, 
                                                  transforms=transforms,
                                                  max_box_num=23, # 
                                                  max_pair_num=num_max_pair, # enough large
                                                  padding=True)
    # load なんかうまくいっていないときがある？？とりあえず再ロードで対策。なんでだ。
    model.model.trainable = True
    model.model.load_weights(load_path)
    model.model.trainable = False
    return model, preprocessor


def get_foldcnntrain_set(train_df, train_fold=[0,1,2], val_fold=[3], model_type="A", not_first_build=False):
    # get game_play
    fold_info = pd.read_csv(f"{cfg.KMAT_PATH}/output/game_fold.csv")
    fold_01_game = fold_info.loc[np.logical_or(fold_info["fold"]==0, fold_info["fold"]==1), "game"].values
    fold_23_game = fold_info.loc[np.logical_or(fold_info["fold"]==2, fold_info["fold"]==3), "game"].values
    
    # fold_train_game = fold_info.loc[np.any([fold_info["fold"]==i for i in train_fold], axis=0), "game"].values
    fold_val_game = fold_info.loc[np.any([fold_info["fold"]==i for i in val_fold], axis=0), "game"].values
    
    game_play_names = list(train_df["game_play"].unique())
    game_names = [int(gp.rsplit("_",1)[0]) for gp in game_play_names]
    mask_fold_val = [name in fold_val_game for name in game_names]
    val_set = np.array(game_play_names)[np.array(mask_fold_val)]
    
    train_title = "fold"
    for f_num in sorted(train_fold):
        train_title += str(f_num)
    
    print(f"train by {train_title}, val by {len(val_set)}")
    if model_type == "A":
        print("use itsumono model")
        load_path = f"{cfg.KMAT_PATH}/model/weights/ex000_contdet_run070_{train_title}train_72crop6cbr_sc_mappretrain/final_weights.h5"
    elif model_type == "B":
        print("use crop model (for ensemble?)")
        load_path = f"{cfg.KMAT_PATH}/model/weights/ex000_contdet_run073_{train_title}train_160cropfirst_detpretrain/final_weights.h5"
        #load_path = f"{cfg.KMAT_PATH}/model_192/weights/ex000_contdet_run074_{train_title}train_192cropfirst_detpretrain/final_weights.h5"
    else:
        raise Exception("select from A, B ")
    input_shape=(704, 1280, 3)
    output_shape=(352, 640)
    num_max_pair = train_df.groupby(["game_play", "step"])["nfl_player_id_2"].size().max()
    print(f"cnn model predict {num_max_pair} pairs at max")
    model, preprocessor = build_model(input_shape, output_shape, load_path, num_max_pair, skip_build_map_model=not_first_build)
    return model, preprocessor, val_set


class CNNEnsembler:
    def __init__(self, models, num_output_items=2):
        self.models = models
        self.num_models = len(models)
        self.num_output_items = num_output_items
    
    def predict(self, inputs):
        predictions = [[] for _ in range(self.num_output_items)]
        for model in self.models:
            preds, inputs = model.predict(inputs)
            if len(preds)!=self.num_output_items:
                raise Exception("set num_output_item correctly")
            for i in range(self.num_output_items):
                predictions[i].append(preds[i])
        predictions = [tf.reduce_mean(tf.stack(preds), axis=0) for preds in predictions]
        return predictions, inputs # inputs include positions, (output mapping model)
        

from model.model import matthews_correlation_fixed

# 0219 update
def pred_by_cnn(train_df, tr_tracking, tr_helmets, tr_video_metadata, game_playes_to_pred, model, preprocessor, is_train_dataset=True):

    only_center_frame_of_step = ~cfg.KMAT_CNN_ALL_FRAME
    batch_size = 4
    draw_pred = False

    if only_center_frame_of_step: # every six frames(60fps->10fps)
        frame_interval = 1 # must be set 1
    else:
        frame_interval = 1 # 中央フレームだけでは見えにくいフレームもあると思うので、理想的には1step予測に数フレーム使う方がいいと思う。

    df_cnn_preds_end = []    
    df_cnn_preds_side = []    
    df_map_preds_end = []    
    df_map_preds_side = []    
    for game_play in game_playes_to_pred:#labels_mini["game_play"].unique():game_play_names
        print(game_play)
        for view in ["Sideline", "Endzone"]:
            print(gc.collect())
            gp_labs, gp_helms = prepare_cnn_dataframe(game_play, train_df, tr_helmets, 
                                            tr_video_metadata, view,
                                           )
            if is_train_dataset:
                video_path = f"{cfg.INPUT}/train/{game_play}_{view}.mp4"
            else:
                video_path = f"{cfg.INPUT}/test/{game_play}_{view}.mp4"
                
            game_play_view = f"{game_play}_{view}"

            vi = Video2Input(game_play_view,
                        video_path,
                         gp_labs,   gp_helms,      
                        frame_interval=frame_interval)

            stacked_inputs = DataStacker(batch_size)
            stacked_targets = DataStacker(batch_size)
            stacked_info = DataStacker(batch_size, as_list=False)
            start_time = time.time()
            counter = 0
            d_counter = 0
            predicted_labels = []
            gt_labels = []
            players_1 = []
            players_2 = []
            step_numbers = []
            frame_numbers = []
            
            map_positions =[]
            map_player_id = []
            player_single_contacts = []
            step_numbers_map = []
            frame_numbers_map = []
                
            is_last_batch = False
            while True:
                step_frame_no, data = vi.get_next(only_center_frame_of_step=only_center_frame_of_step)

                if data is None:
                    is_last_batch = True
                    is_ready = True
                    if stacked_inputs.length==0:
                        break
                else:
                    d_counter += 1

                    try:
                        inputs, targets, info = preprocessor(data)
                        #今少し良くないやり方.要修正、このプリ処理で消えるやつがおる。数変わるのでちゅうい
                    except:# クロップレンジ外。一時的
                        print("ラベル無し")
                        continue
                    step_numbers += [step_frame_no[0]] * int(info["num_labels"])
                    frame_numbers += [step_frame_no[1]] * int(info["num_labels"])
                    
                    step_numbers_map += [step_frame_no[0]] * int(info["num_player"])
                    frame_numbers_map += [step_frame_no[1]] * int(info["num_player"])


                    stacked_inputs.add(inputs)
                    stacked_targets.add(targets)
                    stacked_info.add(info)
                    is_ready = stacked_inputs.is_ready()

                if is_ready:

                    _, inp = stacked_inputs.get_if_ready(neglect_readiness=is_last_batch)
                    _, targ = stacked_targets.get_if_ready(neglect_readiness=is_last_batch)
                    _, info = stacked_info.get_if_ready(neglect_readiness=is_last_batch)
                    
                    #for key in inp.keys():
                    #    print(key, inp[key].shape)
                    #    if key=="input_boxes":
                    #        print(inp[key])
                    #    if key=="input_pairs":
                    #        print(inp[key])
                        
                    preds, inp = model.predict(inp) # inputs include positions, (output mapping model)
                    pred_mask, pred_label, pred_single_contact = preds # pred_label_wo_map is not using. old output
                    # pred_mask, pred_label, _ = preds # pred_label_wo_map is not using. old output
                    
                    
                    if draw_pred:
                        view_contact_mask(inp["input_rgb"].numpy()[0], 
                                          inp["input_pairs"].numpy()[0], 
                                              pred_mask.numpy()[0, :, :, :, 0], 
                                              pred_label.numpy()[0], 
                                              gt_label=targ["output_contact_label"].numpy()[0],
                                              title_epoch="")

                    for i, [p, gt, num, pairs] in enumerate(zip(pred_label.numpy(), targ["output_contact_label"].numpy(), 
                                                 info["num_labels"], info["contact_pairlabels"])):
                        predicted_labels += list(p[:num])
                        gt_labels += list(gt[:num])
                        players_1 += list(pairs[:num,0].numpy())
                        players_2 += list(pairs[:num,1].numpy())
                        
                    for i, [pos, ps_con, pid, num] in enumerate(zip(inp["input_player_positions"].numpy(), pred_single_contact.numpy(),
                                                            info["player_id"].numpy(), info["num_player"])):
                        map_positions += list(pos[:num])
                        map_player_id += list(pid[:num])
                        player_single_contacts += list(ps_con[:num])
                        
                    counter += batch_size
                    time_elapsed = time.time() - start_time
                    fps_inference = counter / time_elapsed
                    print(f"\r{round(fps_inference, 1)} fps, at {vi.current_frame} / {vi.total_frames} in video", end="")
                    
                    
                if is_last_batch:
                    break
           
            df_pred = pd.DataFrame(np.array(predicted_labels).reshape(-1,1), columns=[f"cnn_pred_{view}"])
            df_pred["nfl_player_id_1"] = players_1
            df_pred["nfl_player_id_2"] = players_2
            df_pred["game_play"] = game_play
            df_pred["step"] = step_numbers
            df_pred["frame"] = frame_numbers
            df_pred["gt_tmp"] = gt_labels
            
            df_pred_map = pd.DataFrame(np.array(map_positions).reshape(-1,2), columns=[f"pred_coords_i_{view}", f"pred_coords_j_{view}"])
            df_pred_map[f"player_single_contacts_{view}"] = player_single_contacts
            df_pred_map["nfl_player_id"] = map_player_id
            df_pred_map["step"] = step_numbers_map
            df_pred_map["frame"] = frame_numbers_map
            df_pred_map["game_play"] = game_play
            # df_pred = df_pred.groupby(["step", "game_play", "nfl_player_id_1","nfl_player_id_2"]).mean().reset_index()
            # min, max, meanなど後で処理する
            if view == "Sideline":
                df_cnn_preds_side.append(df_pred)
                df_map_preds_side.append(df_pred_map)
            else:
                df_cnn_preds_end.append(df_pred)
                df_map_preds_end.append(df_pred_map)

            show_each_matthews = True
            if show_each_matthews:
                gt_labels = tf.cast(gt_labels, tf.float32)
                predicted_labels = tf.cast(predicted_labels, tf.float32)
                for th in np.linspace(0.1, 0.9, 9):
                    print(th, matthews_correlation_fixed(gt_labels, predicted_labels, threshold=th))

    df_cnn_preds_end = pd.concat(df_cnn_preds_end, axis=0)
    df_cnn_preds_side = pd.concat(df_cnn_preds_side, axis=0)
    df_map_preds_end = pd.concat(df_map_preds_end, axis=0)
    df_map_preds_side = pd.concat(df_map_preds_side, axis=0)
    print(gc.collect())
    return df_cnn_preds_end, df_cnn_preds_side, df_map_preds_end, df_map_preds_side

# 0219 update
def pred_by_cnn_for_ensemble(train_df, tr_tracking, tr_helmets, tr_video_metadata, game_playes_to_pred, models, preprocessor, is_train_dataset=True):
    """
    models = [CNNEnsembler, CNNEnsembler] or [model, model]
    output 'cnn_pred_{view}_{model_no}'
    """
    num_models = len(models)

    only_center_frame_of_step = ~cfg.KMAT_CNN_ALL_FRAME
    batch_size = 4
    draw_pred = False

    if only_center_frame_of_step: # every six frames(60fps->10fps)
        frame_interval = 1 # must be set 1
    else:
        frame_interval = 1 # 中央フレームだけでは見えにくいフレームもあると思うので、理想的には1step予測に数フレーム使う方がいいと思う。

    df_cnn_preds_end = []    
    df_cnn_preds_side = []    
    df_map_preds_end = []    
    df_map_preds_side = []    
    for game_play in game_playes_to_pred:#labels_mini["game_play"].unique():game_play_names
        print(game_play)
        for view in ["Sideline", "Endzone"]:
            print(gc.collect())
            gp_labs, gp_helms = prepare_cnn_dataframe(game_play, train_df, tr_helmets, 
                                            tr_video_metadata, view,
                                           )
            if is_train_dataset:
                video_path = f"{cfg.INPUT}/train/{game_play}_{view}.mp4"
            else:
                video_path = f"{cfg.INPUT}/test/{game_play}_{view}.mp4"
                
            game_play_view = f"{game_play}_{view}"

            vi = Video2Input(game_play_view,
                        video_path,
                         gp_labs,   gp_helms,      
                        frame_interval=frame_interval)

            stacked_inputs = DataStacker(batch_size)
            stacked_targets = DataStacker(batch_size)
            stacked_info = DataStacker(batch_size, as_list=False)
            start_time = time.time()
            counter = 0
            d_counter = 0
            predicted_labels = [[] for _ in range(num_models)]
            gt_labels = []
            players_1 = []
            players_2 = []
            step_numbers = []
            frame_numbers = []
            
            map_positions =[]
            map_player_id = []
            player_single_contacts = [[] for _ in range(num_models)]
            step_numbers_map = []
            frame_numbers_map = []
                
            is_last_batch = False
            while True:
                step_frame_no, data = vi.get_next(only_center_frame_of_step=only_center_frame_of_step)

                if data is None:
                    is_last_batch = True
                    is_ready = True
                    if stacked_inputs.length==0:
                        break
                else:
                    d_counter += 1

                    try:
                        inputs, targets, info = preprocessor(data)
                        #今少し良くないやり方.要修正、このプリ処理で消えるやつがおる。数変わるのでちゅうい
                    except:# クロップレンジ外。一時的
                        print("ラベル無し")
                        continue
                    step_numbers += [step_frame_no[0]] * int(info["num_labels"])
                    frame_numbers += [step_frame_no[1]] * int(info["num_labels"])
                    
                    step_numbers_map += [step_frame_no[0]] * int(info["num_player"])
                    frame_numbers_map += [step_frame_no[1]] * int(info["num_player"])


                    stacked_inputs.add(inputs)
                    stacked_targets.add(targets)
                    stacked_info.add(info)
                    is_ready = stacked_inputs.is_ready()

                if is_ready:

                    _, inp = stacked_inputs.get_if_ready(neglect_readiness=is_last_batch)
                    _, targ = stacked_targets.get_if_ready(neglect_readiness=is_last_batch)
                    _, info = stacked_info.get_if_ready(neglect_readiness=is_last_batch)
                    
                    #for key in inp.keys():
                    #    print(key, inp[key].shape)
                    #    if key=="input_boxes":
                    #        print(inp[key])
                    #    if key=="input_pairs":
                    #        print(inp[key])
                    
                    list_pred_label = []
                    list_pred_single_contact = []
                    for model in models:
                        preds, inp = model.predict(inp) # inputs include positions, (output mapping model) after first prediction
                        list_pred_label.append(preds[1])
                        list_pred_single_contact.append(preds[2])
                    # pred_mask, pred_label, pred_single_contact = preds # pred_label_wo_map is not using. old output
                    # pred_mask, pred_label, _ = preds # pred_label_wo_map is not using. old output
                    list_pred_label = tf.stack(list_pred_label, axis=1).numpy()
                    list_pred_single_contact = tf.stack(list_pred_single_contact, axis=1).numpy()
                    for i, [p, gt, num, pairs] in enumerate(zip(list_pred_label, targ["output_contact_label"].numpy(), 
                                                 info["num_labels"], info["contact_pairlabels"])):
                        for j in range(num_models):
                            predicted_labels[j] += list(p[j,:num])
                        gt_labels += list(gt[:num])
                        players_1 += list(pairs[:num,0].numpy())
                        players_2 += list(pairs[:num,1].numpy())
                        
                    for i, [pos, ps_con, pid, num] in enumerate(zip(inp["input_player_positions"].numpy(), list_pred_single_contact,
                                                            info["player_id"].numpy(), info["num_player"])):
                        map_positions += list(pos[:num])
                        map_player_id += list(pid[:num])
                        for j in range(num_models):
                            player_single_contacts[j] += list(ps_con[j,:num])
                        
                    counter += batch_size
                    time_elapsed = time.time() - start_time
                    fps_inference = counter / time_elapsed
                    print(f"\r{round(fps_inference, 1)} fps, at {vi.current_frame} / {vi.total_frames} in video", end="")
                    
                    
                if is_last_batch:
                    break
            print("shape of prediction ", np.array(predicted_labels).shape, np.array(player_single_contacts).shape)
            df_pred = pd.DataFrame(np.array(predicted_labels).T, columns=[f"cnn_pred_{view}_{model_no}" for model_no in range(num_models)])
            df_pred["nfl_player_id_1"] = players_1
            df_pred["nfl_player_id_2"] = players_2
            df_pred["game_play"] = game_play
            df_pred["step"] = step_numbers
            df_pred["frame"] = frame_numbers
            df_pred["gt_tmp"] = gt_labels
            
            df_pred_map = pd.DataFrame(np.array(map_positions).reshape(-1,2), columns=[f"pred_coords_i_{view}", f"pred_coords_j_{view}"])
            df_pred_map[[f"player_single_contacts_{view}_{model_no}" for model_no in range(num_models)]] = np.array(player_single_contacts).T
            df_pred_map["nfl_player_id"] = map_player_id
            df_pred_map["step"] = step_numbers_map
            df_pred_map["frame"] = frame_numbers_map
            df_pred_map["game_play"] = game_play
            # df_pred = df_pred.groupby(["step", "game_play", "nfl_player_id_1","nfl_player_id_2"]).mean().reset_index()
            # min, max, meanなど後で処理する
            if view == "Sideline":
                df_cnn_preds_side.append(df_pred)
                df_map_preds_side.append(df_pred_map)
            else:
                df_cnn_preds_end.append(df_pred)
                df_map_preds_end.append(df_pred_map)

            show_each_matthews = True
            if show_each_matthews:
                gt_labels = tf.cast(gt_labels, tf.float32)
                predicted_labels = tf.reduce_mean(tf.cast(predicted_labels, tf.float32), axis=0)
                for th in np.linspace(0.1, 0.9, 9):
                    print(th, matthews_correlation_fixed(gt_labels, predicted_labels, threshold=th))

    df_cnn_preds_end = pd.concat(df_cnn_preds_end, axis=0)
    df_cnn_preds_side = pd.concat(df_cnn_preds_side, axis=0)
    df_map_preds_end = pd.concat(df_map_preds_end, axis=0)
    df_map_preds_side = pd.concat(df_map_preds_side, axis=0)
    print(gc.collect())
    return df_cnn_preds_end, df_cnn_preds_side, df_map_preds_end, df_map_preds_side

def postprocess_cnn_all_frame_outputs(df_pred):
    # 追加。フレーム単位出力の場合、この辺で適当に処理を加える
    # ここで追加したものは、後段のcnn_features関数でマージされる。特別指定しない限りは追加したものは全てマージされる（はず
    # df_predのcolumnsは"game_play", "step", "frame", "nfl_player_id_1", "nfl_player_id_2", "cnn_pred_Sideline" or "cnn_pred_Endzone", "gt_tmp"
    # TODO gt_tmpはゴミ。捨てる。cnn_predの名称統一した方が便利かも…。
    df_pred = df_pred.groupby(["step", "game_play", "nfl_player_id_1","nfl_player_id_2"]).mean().reset_index()
    return df_pred

# 0222 update
def cnn_features_val(train_df, tr_tracking, tr_helmets, tr_video_metadata, cnn_pred_path=f"{cfg.INPUT_DIR}/nfl2cnnpred1218", model_type="B"):
    if cfg.KMAT_CNN_ALL_FRAME: # load all outputs
        suffix = "_raw"
    else:
        suffix = ""
    
    if "distance" in train_df.columns:
        dist_btw_players = train_df["distance"].copy()
    else:
        dist_btw_players = distance(train_df["x_position_1"], train_df["y_position_1"], train_df["x_position_2"], train_df["y_position_2"])
    dist_thresh = 3
    ground_id = train_df["nfl_player_id_2"].min()
    train_df_mini = train_df[np.logical_or(dist_btw_players<dist_thresh, train_df["nfl_player_id_2"]==ground_id)].copy()
    train_df_mini['nfl_player_id_2'] = train_df_mini['nfl_player_id_2'].replace(ground_id,0).astype(int)
    
    if os.path.exists(cnn_pred_path):
        print("load existing pred file")
        print(f"fold01_cnn_pred_end{suffix}.csv")
        
        df_side = []
        df_end = []
        for i in range(4):
            df_cnn_preds_end = pd.read_csv(os.path.join(cnn_pred_path, f"fold{i}_cnn_pred_end{suffix}.csv"))
            df_cnn_preds_side = pd.read_csv(os.path.join(cnn_pred_path, f"fold{i}_cnn_pred_side{suffix}.csv"))
            df_cnn_preds_end[["nfl_player_id_1", "nfl_player_id_2"]] = df_cnn_preds_end[["nfl_player_id_1", "nfl_player_id_2"]].astype(int)
            df_cnn_preds_side[["nfl_player_id_1", "nfl_player_id_2"]] = df_cnn_preds_side[["nfl_player_id_1", "nfl_player_id_2"]].astype(int)
            df_end.append(df_cnn_preds_end)
            df_side.append(df_cnn_preds_side)
        df_side = pd.concat(df_side, axis=0)
        df_end = pd.concat(df_end, axis=0)

        df_end_map = []
        for i in range(4):
            df_end_map.append(pd.read_csv(os.path.join(cnn_pred_path, f"fold{i}_map_pred_end{suffix}.csv")))
        df_end_map = pd.concat(df_end_map, axis=0)
        df_end_map["nfl_player_id"] = df_end_map["nfl_player_id"].astype(int)

        df_side_map = []
        for i in range(4):
            df_side_map.append(pd.read_csv(os.path.join(cnn_pred_path, f"fold{i}_map_pred_side{suffix}.csv")))
        df_side_map = pd.concat(df_side_map, axis=0)
        df_side_map["nfl_player_id"] = df_side_map["nfl_player_id"].astype(int)
               
    else:
        print("start CNN validation pred")
        K.clear_session()
        
        os.makedirs(cnn_pred_path, exist_ok=True)

        train_fold=[1,2,3]
        val_fold=[0]
        model, preprocessor, val_set = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type)
        df_cnn_preds_end_0, df_cnn_preds_side_0, df_map_preds_end_0, df_map_preds_side_0 = pred_by_cnn(train_df_mini, tr_tracking, tr_helmets, tr_video_metadata, val_set, model, preprocessor, is_train_dataset=True)
        df_cnn_preds_end_0.to_csv(os.path.join(cnn_pred_path, f"fold0_cnn_pred_end{suffix}.csv"), index=False)
        df_cnn_preds_side_0.to_csv(os.path.join(cnn_pred_path, f"fold0_cnn_pred_side{suffix}.csv"), index=False)
        df_map_preds_end_0.to_csv(os.path.join(cnn_pred_path, f"fold0_map_pred_end{suffix}.csv"), index=False)
        df_map_preds_side_0.to_csv(os.path.join(cnn_pred_path, f"fold0_map_pred_side{suffix}.csv"), index=False)
        
        train_fold=[0,2,3]
        val_fold=[1]
        model, preprocessor, val_set = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type)
        df_cnn_preds_end_1, df_cnn_preds_side_1, df_map_preds_end_1, df_map_preds_side_1 = pred_by_cnn(train_df_mini, tr_tracking, tr_helmets, tr_video_metadata, val_set, model, preprocessor, is_train_dataset=True)
        df_cnn_preds_end_1.to_csv(os.path.join(cnn_pred_path, f"fold1_cnn_pred_end{suffix}.csv"), index=False)
        df_cnn_preds_side_1.to_csv(os.path.join(cnn_pred_path, f"fold1_cnn_pred_side{suffix}.csv"), index=False)
        df_map_preds_end_1.to_csv(os.path.join(cnn_pred_path, f"fold1_map_pred_end{suffix}.csv"), index=False)
        df_map_preds_side_1.to_csv(os.path.join(cnn_pred_path, f"fold1_map_pred_side{suffix}.csv"), index=False)
        
        
        train_fold=[0,1,3]
        val_fold=[2]
        model, preprocessor, val_set = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type)
        df_cnn_preds_end_2, df_cnn_preds_side_2, df_map_preds_end_2, df_map_preds_side_2 = pred_by_cnn(train_df_mini, tr_tracking, tr_helmets, tr_video_metadata, val_set, model, preprocessor, is_train_dataset=True)
        df_cnn_preds_end_2.to_csv(os.path.join(cnn_pred_path, f"fold2_cnn_pred_end{suffix}.csv"), index=False)
        df_cnn_preds_side_2.to_csv(os.path.join(cnn_pred_path, f"fold2_cnn_pred_side{suffix}.csv"), index=False)
        df_map_preds_end_2.to_csv(os.path.join(cnn_pred_path, f"fold2_map_pred_end{suffix}.csv"), index=False)
        df_map_preds_side_2.to_csv(os.path.join(cnn_pred_path, f"fold2_map_pred_side{suffix}.csv"), index=False)
        
        train_fold=[0,1,2]
        val_fold=[3]
        model, preprocessor, val_set = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type)
        df_cnn_preds_end_3, df_cnn_preds_side_3, df_map_preds_end_3, df_map_preds_side_3 = pred_by_cnn(train_df_mini, tr_tracking, tr_helmets, tr_video_metadata, val_set, model, preprocessor, is_train_dataset=True)
        df_cnn_preds_end_3.to_csv(os.path.join(cnn_pred_path, f"fold3_cnn_pred_end{suffix}.csv"), index=False)
        df_cnn_preds_side_3.to_csv(os.path.join(cnn_pred_path, f"fold3_cnn_pred_side{suffix}.csv"), index=False)
        df_map_preds_end_3.to_csv(os.path.join(cnn_pred_path, f"fold3_map_pred_end{suffix}.csv"), index=False)
        df_map_preds_side_3.to_csv(os.path.join(cnn_pred_path, f"fold3_map_pred_side{suffix}.csv"), index=False)
        

        df_side = pd.concat([df_cnn_preds_side_0, df_cnn_preds_side_1, df_cnn_preds_side_2, df_cnn_preds_side_3], axis=0)
        df_end = pd.concat([df_cnn_preds_end_0, df_cnn_preds_end_1, df_cnn_preds_end_2, df_cnn_preds_end_3], axis=0)
        df_side_map = pd.concat([df_map_preds_side_0, df_map_preds_side_1, df_map_preds_side_2, df_map_preds_side_3], axis=0)
        df_end_map = pd.concat([df_map_preds_end_0, df_map_preds_end_1, df_map_preds_end_2, df_map_preds_end_3], axis=0)
        
    # 追加。フレーム単位出力の場合、この辺で適当に処理を加える
    # ここで追加したものは、後段cnn_featuresでマージする
    df_side = postprocess_cnn_all_frame_outputs(df_side)
    df_end = postprocess_cnn_all_frame_outputs(df_end)
    
    df_end['nfl_player_id_2'] = df_end['nfl_player_id_2'].replace(0,ground_id).astype(int)
    df_side['nfl_player_id_2'] = df_side['nfl_player_id_2'].replace(0,ground_id).astype(int)

    return df_side, df_end, df_side_map, df_end_map

# 0219 update
def cnn_features_test(train_df, tr_tracking, tr_helmets, tr_video_metadata, model_type="B"):
    print(gc.collect())
    save_path_end = "cnn_pred_end.csv"
    save_path_side = "cnn_pred_side.csv"
    save_path_map_end = "map_pred_end.csv"
    save_path_map_side = "map_pred_side.csv"
    if os.path.exists(save_path_end):
        df_cnn_preds_end = pd.read_csv(save_path_end)
        df_cnn_preds_side = pd.read_csv(save_path_side)
        df_end_map = pd.read_csv(save_path_map_end)
        df_side_map = pd.read_csv(save_path_map_side)
        df_cnn_preds_end[["nfl_player_id_1", "nfl_player_id_2"]] = df_cnn_preds_end[["nfl_player_id_1", "nfl_player_id_2"]].astype(int)
        df_cnn_preds_side[["nfl_player_id_1", "nfl_player_id_2"]] = df_cnn_preds_side[["nfl_player_id_1", "nfl_player_id_2"]].astype(int)
        df_end_map["nfl_player_id"] = df_end_map["nfl_player_id"].astype(int)
        df_side_map["nfl_player_id"] = df_side_map["nfl_player_id"].astype(int)
    else:
        if "distance" in train_df.columns:
            dist_btw_players = train_df["distance"].copy()
        else:
            dist_btw_players = distance(df["x_position_1"], df["y_position_1"], df["x_position_2"], df["y_position_2"])

        dist_thresh = 3
        ground_id = train_df["nfl_player_id_2"].min() # kmat modelでは0を使用していたのでid No調整。関数出る前に戻す。
        train_df_mini = train_df[np.logical_or(dist_btw_players<dist_thresh, train_df["nfl_player_id_2"]==ground_id)].copy()
        train_df_mini['nfl_player_id_2'] = train_df_mini['nfl_player_id_2'].replace(ground_id,0).astype(int)
        train_df_mini["contact"] = 1. # not use

        game_plays = list(train_df_mini["game_play"].unique())
        # model_01, preprocessor, _ = get_foldcnntrain_set(train_df, train_01_val_23 = True)
        # model_23, preprocessor, _ = get_foldcnntrain_set(train_df, train_01_val_23 = False)

        train_fold, val_fold = [1,2,3], [0]
        model_0, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type)
        train_fold, val_fold = [0,2,3], [1]
        model_1, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        train_fold, val_fold = [0,1,3], [2]
        model_2, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        train_fold, val_fold = [0,1,2], [3]
        model_3, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        model = CNNEnsembler([model_0, model_1, model_2, model_3], num_output_items=3)

        df_end, df_side, df_end_map, df_side_map = pred_by_cnn(train_df_mini, tr_tracking, tr_helmets, tr_video_metadata, game_plays, model, preprocessor, is_train_dataset=False)
        df_end.to_csv(save_path_end, index=False)
        df_side.to_csv(save_path_side, index=False)
        df_end_map.to_csv(save_path_map_end, index=False)
        df_side_map.to_csv(save_path_map_side, index=False)

    print(gc.collect())
    df_side = postprocess_cnn_all_frame_outputs(df_side)
    df_end = postprocess_cnn_all_frame_outputs(df_end)

    df_end['nfl_player_id_2'] = df_end['nfl_player_id_2'].replace(0,ground_id).astype(int)
    df_side['nfl_player_id_2'] = df_side['nfl_player_id_2'].replace(0,ground_id).astype(int)

    return df_side, df_end, df_side_map, df_end_map

# 0222 update
def cnn_features_test_ensemble(train_df, tr_tracking, tr_helmets, tr_video_metadata, model_no_to_use, model_type="B"):
    """
    to load the outputs of first model, set model_no_to_use = 0
    
    """
    print(gc.collect())
    save_path_end = "cnn_pred_end.csv"
    save_path_side = "cnn_pred_side.csv"
    save_path_map_end = "map_pred_end.csv"
    save_path_map_side = "map_pred_side.csv"
    if os.path.exists(save_path_end):
        ground_id = -1
        print("read existing files")
        df_end = pd.read_csv(save_path_end)
        df_side = pd.read_csv(save_path_side)
        df_end_map = pd.read_csv(save_path_map_end)
        df_side_map = pd.read_csv(save_path_map_side)
        df_end[["nfl_player_id_1", "nfl_player_id_2"]] = df_end[["nfl_player_id_1", "nfl_player_id_2"]].astype(int)
        df_side[["nfl_player_id_1", "nfl_player_id_2"]] = df_side[["nfl_player_id_1", "nfl_player_id_2"]].astype(int)
        df_end_map["nfl_player_id"] = df_end_map["nfl_player_id"].astype(int)
        df_side_map["nfl_player_id"] = df_side_map["nfl_player_id"].astype(int)
    else:
        if "distance" in train_df.columns:
            dist_btw_players = train_df["distance"].copy()
        else:
            dist_btw_players = distance(df["x_position_1"], df["y_position_1"], df["x_position_2"], df["y_position_2"])

        dist_thresh = 3
        ground_id = train_df["nfl_player_id_2"].min() # kmat modelでは0を使用していたのでid No調整。関数出る前に戻す。
        train_df_mini = train_df[np.logical_or(dist_btw_players<dist_thresh, train_df["nfl_player_id_2"]==ground_id)].copy()
        train_df_mini['nfl_player_id_2'] = train_df_mini['nfl_player_id_2'].replace(ground_id,0).astype(int)
        train_df_mini["contact"] = 1. # not use

        game_plays = list(train_df_mini["game_play"].unique())
        # model_01, preprocessor, _ = get_foldcnntrain_set(train_df, train_01_val_23 = True)
        # model_23, preprocessor, _ = get_foldcnntrain_set(train_df, train_01_val_23 = False)
        """
        
        model_type = "A"
        train_fold, val_fold = [1,2,3], [0]
        model_0, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type)
        train_fold, val_fold = [0,2,3], [1]
        model_1, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        train_fold, val_fold = [0,1,3], [2]
        model_2, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        train_fold, val_fold = [0,1,2], [3]
        model_3, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        model_A = CNNEnsembler([model_0, model_1, model_2, model_3], num_output_items=3)
        """
        train_fold, val_fold = [1,2,3], [0]
        
        model_0, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type)
        train_fold, val_fold = [0,2,3], [1]
        model_1, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        train_fold, val_fold = [0,1,3], [2]
        model_2, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        train_fold, val_fold = [0,1,2], [3]
        model_3, preprocessor, _ = get_foldcnntrain_set(train_df_mini, train_fold, val_fold, model_type=model_type, not_first_build=True)
        # model_B = CNNEnsembler([model_0, model_1, model_2, model_3], num_output_items=3)
        
        models = [model_0, model_1, model_2, model_3]

        # model = CNNEnsembler([model_0], num_output_items=3)
        print(gc.collect())

        # df_end, df_side, df_end_map, df_side_map = pred_by_cnn(train_df_mini, tr_tracking, tr_helmets, tr_video_metadata, game_plays, model, preprocessor, is_train_dataset=False)
        df_end, df_side, df_end_map, df_side_map = pred_by_cnn_for_ensemble(train_df_mini, tr_tracking, tr_helmets, tr_video_metadata, game_plays, models, preprocessor, is_train_dataset=False)
        df_end.to_csv(save_path_end, index=False)
        df_side.to_csv(save_path_side, index=False)
        df_end_map.to_csv(save_path_map_end, index=False)
        df_side_map.to_csv(save_path_map_side, index=False)

    print(gc.collect())
    # select output columns and rename
    print(df_end.columns)
    view = "Endzone"
    #'cnn_pred_Endzone_0'
    columns_pair = [f"cnn_pred_{view}_{model_no_to_use}", "nfl_player_id_1", "nfl_player_id_2","game_play","step","frame"]
    columns_single = [f"pred_coords_i_{view}", f"pred_coords_j_{view}",f"player_single_contacts_{view}_{model_no_to_use}","nfl_player_id","step","frame","game_play"]
    df_end = df_end[columns_pair].rename(columns={f"cnn_pred_{view}_{model_no_to_use}": f"cnn_pred_{view}"})
    df_end_map = df_end_map[columns_single].rename(columns={f"player_single_contacts_{view}_{model_no_to_use}": f"player_single_contacts_{view}"})
    print(df_end.columns)
    
    view = "Sideline"
    columns_pair = [f"cnn_pred_{view}_{model_no_to_use}", "nfl_player_id_1", "nfl_player_id_2","game_play","step","frame"]
    columns_single = [f"pred_coords_i_{view}", f"pred_coords_j_{view}",f"player_single_contacts_{view}_{model_no_to_use}","nfl_player_id","step","frame","game_play"]
    df_side = df_side[columns_pair].rename(columns={f"cnn_pred_{view}_{model_no_to_use}": f"cnn_pred_{view}"})
    df_side_map = df_side_map[columns_single].rename(columns={f"player_single_contacts_{view}_{model_no_to_use}": f"player_single_contacts_{view}"})
    print(gc.collect())
    df_end = postprocess_cnn_all_frame_outputs(df_end)
    df_side = postprocess_cnn_all_frame_outputs(df_side)
    
    df_end['nfl_player_id_2'] = df_end['nfl_player_id_2'].replace(0,ground_id).astype(int)
    df_side['nfl_player_id_2'] = df_side['nfl_player_id_2'].replace(0,ground_id).astype(int)

    return df_side, df_end, df_side_map, df_end_map

## points set matching(kmat)

検出対象外も含めて座標ワープする。(フレーム内にいるはずなのに見えていないのか、フレーム外にいるか、を明確に。)

- prepare_matching_dataframeのmergeをright(tracking)側に。
- 検出箱付きとは別に全プレイヤを一旦残して、出力された変換行列を用いてワープ。

In [ ]:
"""
points set matching.
run 1 play at once.
"""

def batch_points2points_fitting(targets, sources, padded, num_iter=6, l2_reg=0.1, init_rot=0):
    
    def get_transmatrix(kx, ky, rz, tx, ty):
        """
        k : log(zoom ratio).
        rz : rotation.
        tx : x offset.
        ty : z offset
        shape [batch]
        
        returns:
            transmatrix with shape [batch, 3, 3]
        """
        exp_kx = tf.math.exp(kx)
        exp_ky = tf.math.exp(ky)
        
        sin = tf.math.sin(rz)
        cos = tf.math.cos(rz)
        mat = tf.stack([[exp_kx*cos, -exp_ky*sin, 1*tx],
                        [exp_kx*sin, exp_ky*cos, 1*ty],
                        [tf.zeros_like(kx), tf.zeros_like(kx), tf.ones_like(kx)]])
        mat = tf.transpose(mat, [2,0,1])
        return mat
        
    def transform_points(transmatrix, points):
        x, y = tf.split(points, 2, axis=-1)
        xyones = tf.concat([x,y,tf.ones_like(x)], axis=-1)
        trans_points = tf.matmul(xyones, transmatrix, transpose_b=True)[...,:2]
        return trans_points
    
    def get_derivative_at(kx, ky, rz, tx, ty, points):
        dev = 1e-5
        original = transform_points(get_transmatrix(kx, ky, rz, tx, ty), points)
        dxy_dkx = (transform_points(get_transmatrix(kx+dev, ky, rz, tx, ty), points) - original) / dev
        dxy_dky = (transform_points(get_transmatrix(kx, ky+dev, rz, tx, ty), points) - original) / dev
        dxy_drz = (transform_points(get_transmatrix(kx, ky, rz+dev, tx, ty), points) - original) / dev
        dxy_dtx = (transform_points(get_transmatrix(kx, ky, rz, tx+dev, ty), points) - original) / dev
        dxy_dty = (transform_points(get_transmatrix(kx, ky, rz, tx, ty+dev), points) - original) / dev
        return original, dxy_dkx, dxy_dky, dxy_drz, dxy_dtx, dxy_dty
    
    # initial_values
    batch, num_points = tf.unstack(tf.shape(targets))[:2]
    kx = 0.0 * tf.ones((batch), tf.float32)#zoom ratio = exp(k)
    ky = 0.0 * tf.ones((batch), tf.float32)#zoom ratio = exp(k)
    rz = init_rot * tf.ones((batch), tf.float32)
    tx = 0.0 * tf.ones((batch), tf.float32)
    ty = 0.0 * tf.ones((batch), tf.float32)
    
    source_origin = sources#tf.stop_gradient(sources)
    for i in range(num_iter):
        currents, dxy_dkx, dxy_dky, dxy_rz, dxy_dtx, dxy_dty = get_derivative_at(kx, ky, rz, tx, ty, source_origin)
        b = tf.reshape((targets-currents)*padded, [batch, num_points*2, 1])#xy flatten
        a = tf.stack([dxy_dkx, dxy_dky, dxy_rz, dxy_dtx, dxy_dty], axis=-1)
        a = a * padded[...,tf.newaxis]
        a = tf.reshape(a, [batch, num_points*2, 5])
        updates = tf.linalg.lstsq(a, b, l2_regularizer=l2_reg, fast=True)#batch, 5, 1
        kx = tf.maximum(kx + updates[:,0,0], -10)
        ky = tf.maximum(ky + updates[:,1,0], -10)
        rz = rz + updates[:,2,0]
        tx = tx + updates[:,3,0]
        ty = ty + updates[:,4,0]
           
    trans_matrix = get_transmatrix(kx, ky, rz, tx, ty)
    trans_sources = transform_points(trans_matrix, sources)
    try:
        trans_targets = transform_points(tf.linalg.inv(trans_matrix), targets)
    except:
        trans_targets = targets    
    residuals_points = tf.reduce_sum(((targets-trans_sources)*padded)**2, axis=2)
    residuals = tf.reduce_sum(residuals_points, axis=1)
    return trans_sources, trans_targets, trans_matrix, kx, ky, rz, tx, ty, residuals_points, residuals

def batch_points2points_fitting_highdof(targets, sources, padded, num_iter=6, l2_reg=0.1, init_rot=0):
    
    def get_transmatrix(kx, ky, rz, tx, ty, trape):
        """
        k : log(zoom ratio).
        rz : rotation.
        tx : x offset.
        ty : z offset
        shape [batch]
        
        returns:
            transmatrix with shape [batch, 3, 3]
        """
        exp_kx = tf.math.exp(kx)
        exp_ky = tf.math.exp(ky)
        trape = trape#trapezoid
        sin = tf.math.sin(rz)
        cos = tf.math.cos(rz)
        mat = tf.stack([[exp_kx*cos, trape*exp_kx*cos - exp_ky * sin, 1*tx],
                        [exp_kx*sin, trape*exp_kx*sin + exp_ky * cos, 1*ty],
                        [tf.zeros_like(kx), tf.zeros_like(kx), tf.ones_like(kx)]])
        mat = tf.transpose(mat, [2,0,1])
        return mat
        
    def transform_points(transmatrix, points):
        x, y = tf.split(points, 2, axis=-1)
        xyones = tf.concat([x,y,tf.ones_like(x)], axis=-1)
        trans_points = tf.matmul(xyones, transmatrix, transpose_b=True)[...,:2]
        return trans_points
    
    def get_derivative_at(kx, ky, rz, tx, ty, trape, points):
        dev = 1e-5
        original = transform_points(get_transmatrix(kx, ky, rz, tx, ty, trape), points)
        dxy_dkx = (transform_points(get_transmatrix(kx+dev, ky, rz, tx, ty, trape), points) - original) / dev
        dxy_dky = (transform_points(get_transmatrix(kx, ky+dev, rz, tx, ty, trape), points) - original) / dev
        dxy_drz = (transform_points(get_transmatrix(kx, ky, rz+dev, tx, ty, trape), points) - original) / dev
        dxy_dtx = (transform_points(get_transmatrix(kx, ky, rz, tx+dev, ty, trape), points) - original) / dev
        dxy_dty = (transform_points(get_transmatrix(kx, ky, rz, tx, ty+dev, trape), points) - original) / dev
        dxy_dttrape = (transform_points(get_transmatrix(kx, ky, rz, tx, ty, trape+dev), points) - original) / dev
        return original, dxy_dkx, dxy_dky, dxy_drz, dxy_dtx, dxy_dty, dxy_dttrape
    
    # initial_values
    batch, num_points = tf.unstack(tf.shape(targets))[:2]
    kx = 0.0 * tf.ones((batch), tf.float32)#zoom ratio = exp(k)
    ky = 0.0 * tf.ones((batch), tf.float32)#zoom ratio = exp(k)
    rz = init_rot * tf.ones((batch), tf.float32)
    tx = 0.0 * tf.ones((batch), tf.float32)
    ty = 0.0 * tf.ones((batch), tf.float32)
    trape = 0.0 * tf.ones((batch), tf.float32)
    
    source_origin = sources#tf.stop_gradient(sources)
    for i in range(num_iter):
        currents, dxy_dkx, dxy_dky, dxy_rz, dxy_dtx, dxy_dty, dxy_dttrape = get_derivative_at(kx, ky, rz, tx, ty, trape, source_origin)
        b = tf.reshape((targets-currents)*padded, [batch, num_points*2, 1])#xy flatten
        a = tf.stack([dxy_dkx, dxy_dky, dxy_rz, dxy_dtx, dxy_dty, dxy_dttrape], axis=-1)
        a = a * padded[...,tf.newaxis]
        a = tf.reshape(a, [batch, num_points*2, 6])
        updates = tf.linalg.lstsq(a, b, l2_regularizer=l2_reg, fast=True)#batch, 6, 1
        kx = tf.maximum(kx + updates[:,0,0], -10)
        ky = tf.maximum(ky + updates[:,1,0], -10)
        rz = rz + updates[:,2,0]
        tx = tx + updates[:,3,0]
        ty = ty + updates[:,4,0]
        trape = tf.clip_by_value(trape + updates[:,5,0], -0.25, 0.25)#
           
    trans_matrix = get_transmatrix(kx, ky, rz, tx, ty, trape)
    trans_sources = transform_points(trans_matrix, sources)
    try:
        trans_targets = transform_points(tf.linalg.inv(trans_matrix), targets)
    except:
        trans_targets = targets    
    residuals_points = tf.reduce_sum(((targets-trans_sources)*padded)**2, axis=2)
    residuals = tf.reduce_sum(residuals_points, axis=1)
    return trans_sources, trans_targets, trans_matrix, kx, ky, rz, tx, ty, residuals_points, residuals

def transform_xyz_to_img_coords(points, trans_matrix, inverse=True):
    if inverse:
        trans_matrix = tf.linalg.inv(trans_matrix)
    x, y = tf.split(points, 2, axis=-1)
    xyones = tf.concat([x,y,tf.ones_like(x)], axis=-1)
    trans_points = tf.matmul(xyones, trans_matrix, transpose_b=True)[...,:2]
    return trans_points

def possible_misassignment(trans_sources, targets, padded, padded_player_ids, top_n=2):
    """
    batch, num_player, 2(x,y)
    """
    batch, num_player, _ = tf.unstack(tf.shape(targets))
    dists = tf.reduce_sum((targets[:,:,tf.newaxis,:] - trans_sources[:,tf.newaxis,:,:])**2, axis=3)
    padded_area = 1 - padded[:,:,tf.newaxis,0] * padded[:,tf.newaxis,:,0]
    current_asign = tf.eye(num_player, dtype=dists.dtype)[tf.newaxis, :,:]
    dists = dists + (padded_area + current_asign) * 1e7
    argsort = tf.argsort(dists, axis=2)[:,:,:top_n]
    possible_residuals = tf.gather(dists, argsort, batch_dims=2)
    possible_player_ids = tf.gather(padded_player_ids, argsort, batch_dims=1)
    return possible_residuals, possible_player_ids

def pad_for_batch(list_tf_targets, list_tf_sources, list_tf_player_ids):
    num_points = [points.shape[0] for points in list_tf_targets]
    max_num = max(num_points)
    num_pads = [max_num - num for num in num_points]
    list_tf_targets = [tf.pad(points, [[0,pad_num],[0,0]]) for points, pad_num in zip(list_tf_targets, num_pads)]
    list_tf_sources = [tf.pad(points, [[0,pad_num],[0,0]]) for points, pad_num in zip(list_tf_sources, num_pads)]
    list_tf_player_ids = [tf.pad(ids, [[0,pad_num]], constant_values=-2) for ids, pad_num in zip(list_tf_player_ids, num_pads)]
    tf_targets = tf.stack(list_tf_targets)
    tf_sources = tf.stack(list_tf_sources)
    tf_player_ids = tf.stack(list_tf_player_ids)
    tf_pad_bools = tf.cast(tf.stack([tf.range(max_num) < num for num in num_points]), tf_targets.dtype)[:,:,tf.newaxis]
    return tf_targets, tf_sources, tf_player_ids, tf_pad_bools, num_points

def pad_for_batch_single_item(list_tf_targets):
    num_points = [points.shape[0] for points in list_tf_targets]
    max_num = max(num_points)
    num_pads = [max_num - num for num in num_points]
    list_tf_targets = [tf.pad(points, [[0,pad_num],[0,0]]) for points, pad_num in zip(list_tf_targets, num_pads)]
    tf_targets = tf.stack(list_tf_targets)
    tf_pad_bools = tf.cast(tf.stack([tf.range(max_num) < num for num in num_points]), tf_targets.dtype)[:,:,tf.newaxis]
    return tf_targets, tf_pad_bools, num_points

def depad_for_batch(tf_items, num_points):
    list_np_items = [[points[:num] for points, num in zip(val.numpy(), num_points)] for val in tf_items]
    #list_tf_sources = [points[:num] for points, num in zip(tf_sources.numpy(), num_points)]
    return list_np_items

def p2p_registration_features(tr_tracking, tr_helmets, tr_video_metadata, p2p_file = "pathto/p2p_registration_residuals.csv", view_some_results=False, num_possible_assign=2):
    """
    num_possible_assign: 
    アサインメントミスの可能性のあるプレイヤを抽出する。
    具体的には現在アサインされている以外に近いプレイヤを抽出する。同時にもしそのプレイヤであった場合の残差も出す(減残差と比較することで、あとでマスク的に使う)
    """
    if p2p_file is not None and os.path.exists(p2p_file):
        df_p2p_regist = pd.read_csv(p2p_file)
        df_all_players_img_coords = pd.read_csv(p2p_file.replace(".csv", "_img_coords.csv"))
        
    else:
        S = time.time()
        outputs = []
        outputs_all_players_img_coords = []
        for game_play in tr_tracking["game_play"].unique():
            print(game_play)
            for view in ["Sideline", "Endzone"]:
                helm_merged = prepare_matching_dataframe(game_play, tr_tracking, tr_helmets, tr_video_metadata, view, fps=59.94, only_center_of_step=True)
                helm_merged["box_x"] = helm_merged["left"] + helm_merged["width"]/2
                helm_merged["box_y"] = helm_merged["top"] + helm_merged["height"]/2
                
                scale_rate = 10
                list_targets = []
                list_sources = []
                list_flip_target = []
                list_flip_target_all = []
                list_box_centers = []
                player_ids = []
                df_out = []
                df_out_all_players = []
                for frame, df_frame in helm_merged.groupby("frame"):
                    #if len(df_frame)!=22:
                    #    print("")
                    df_frame_bbox = df_frame[~df_frame["box_x"].isna()]
                    
                    box_x = df_frame_bbox["box_x"].values
                    box_y = df_frame_bbox["box_y"].values
                    box_x_mean = box_x.mean()
                    box_y_mean = box_y.mean()

                    pos_x = df_frame_bbox["x_position"].values
                    pos_y = df_frame_bbox["y_position"].values
                    pos_x_mean = pos_x.mean()
                    pos_y_mean = pos_y.mean()
                    
                    pos_x_all_player = df_frame["x_position"].values
                    pos_y_all_player = df_frame["y_position"].values
                    
                    if np.isnan(box_x_mean) or np.isnan(pos_x_mean):
                        #print("nan exist")
                        continue

                    adjusted_bx = (box_x - box_x_mean) / scale_rate
                    adjusted_by = (box_y - box_y_mean) / scale_rate

                    adjusted_px = pos_x - pos_x_mean
                    adjusted_py = pos_y - pos_y_mean

                    adjusted_px_all = pos_x_all_player - pos_x_mean
                    adjusted_py_all = pos_y_all_player - pos_y_mean

                    list_targets += [tf.cast(tf.stack([adjusted_px, adjusted_py], axis=-1), tf.float32)]
                    list_sources += [tf.cast(tf.stack([adjusted_bx, adjusted_by], axis=-1), tf.float32)]

                    list_flip_target += [tf.cast(tf.stack([-adjusted_px, adjusted_py], axis=-1), tf.float32)]
                    list_flip_target_all += [tf.cast(tf.stack([-adjusted_px_all, adjusted_py_all], axis=-1), tf.float32)]
                    list_box_centers += [[box_x_mean, box_y_mean]]

                    player_ids += [tf.cast(df_frame_bbox["nfl_player_id"].values, tf.int32)]

                    df_frame_bbox["average_box_size"] = (np.sqrt(df_frame_bbox["width"] * df_frame_bbox["height"])).mean()
                    df_out.append(df_frame_bbox[["game_play","view","frame","step","nfl_player_id","average_box_size"]].copy())
                    df_out_all_players.append(df_frame[["game_play","view","frame","step","nfl_player_id"]].copy())
                    #print(df_frame[["game_play","view","frame","step","nfl_player_id"]].isna().sum(axis=0))

                tf_targets, tf_sources, tf_player_ids, tf_pad_bools, num_points = pad_for_batch(list_targets, list_sources, player_ids)
                tf_flip_targets, tf_sources, _, tf_pad_bools, num_points = pad_for_batch(list_flip_target, list_sources, player_ids)
                
                tf_flip_targets_all, tf_pad_bools_all, num_points_all = pad_for_batch_single_item(list_flip_target_all) # 追加。全プレイヤ出力する。

                with tf.device('/CPU:0'): # gpu少し怪しい。
                    #result_flip = batch_points2points_fitting(tf_flip_targets, tf_sources, tf_pad_bools, num_iter=30, l2_reg=100., init_rot=0)
                    #result_flip_180 = batch_points2points_fitting(tf_flip_targets, tf_sources, tf_pad_bools, num_iter=30, l2_reg=100., init_rot=3.14)
                    result_flip = batch_points2points_fitting_highdof(tf_flip_targets, tf_sources, tf_pad_bools, num_iter=30, l2_reg=100., init_rot=0)
                    result_flip_180 = batch_points2points_fitting_highdof(tf_flip_targets, tf_sources, tf_pad_bools, num_iter=30, l2_reg=100., init_rot=3.14)
                    
                # take smaller (better fit)
                result_flip_stack = [tf.stack([r1,r2], axis=-1) for r1, r2 in zip(result_flip, result_flip_180)]
                better = tf.argmin(result_flip_stack[-1], axis=-1) # last index is residual
                result_flip = [tf.gather(result, better, batch_dims=1, axis=-1) for result in result_flip_stack]

                flip_residual_median = np.median(result_flip[-1])
                print(f"{game_play}, {view}, residual flip: {flip_residual_median}")

                result = result_flip
                view_target = tf_flip_targets

                trans_sources, trans_targets, trans_matrix, kx, ky, rz, tx, ty, residual_points, residual = result
                
                # transform all players (include non-detected players)
                tf_souce_all_players = transform_xyz_to_img_coords(tf_flip_targets_all, trans_matrix, inverse=True)                
                all_players_img_coords = scale_rate * tf_souce_all_players + tf.cast(list_box_centers, tf.float32)[:,tf.newaxis,:]                
                list_targets, list_sources, list_residual = depad_for_batch([view_target, trans_sources, residual_points],
                                                                            num_points) 
                all_players_img_coords = depad_for_batch([all_players_img_coords], num_points_all)[0] 
               

                tiled_resisual = []
                for r, num_p in zip(residual.numpy(), num_points):
                    tiled_resisual += [r] * num_p 

                df_out = pd.concat(df_out, axis=0)
                df_out[["x_position_offset_on_img", "y_position_offset_on_img"]] = (np.concatenate(list_sources, axis=0) - np.concatenate(list_targets, axis=0)) * scale_rate
                df_out["p2p_registration_residual"] = np.concatenate(list_residual, axis=0)
                df_out["p2p_registration_residual_frame"] = tiled_resisual
                outputs.append(df_out)

                df_out_all_players = pd.concat(df_out_all_players, axis=0)
                df_out_all_players[["img_coords_x", "img_coords_y"]] = np.concatenate(all_players_img_coords, axis=0)
                outputs_all_players_img_coords.append(df_out_all_players)
                if view_some_results:
                    for t, s in zip(list_targets[::50], list_sources[::50]):
                        plt.figure(figsize=(8,5))
                        words = np.arange(len(t))
                        plt.scatter(t[:,0], t[:,1], c="blue", s=50, alpha=0.5)#, c=np.arange(len(t)))#, cmap="gray")#color="blue")
                        for number, [x,y] in enumerate(t): 
                            plt.text(x, y+.8, number, fontsize=9, c="blue")
                            #plt.annotate(number, (x, y), c="blue")
                        plt.scatter(s[:,0], s[:,1], c="red", s=50, alpha=0.5)#, c=np.arange(len(s)))#, cmap="gray")#color="red")
                        for number, [x,y] in enumerate(s): 
                            plt.text(x, y-1.75, number, fontsize=9, c="red")
                            #plt.annotate(number, (x, y), c="red")
                        #plt.text(s[:,0]+.3, s[:,1]+.3, words, fontsize=9)
                        plt.grid()
                        plt.title("tracking X-Y  VS  transformed image coord")
                        plt.show()            
        df_p2p_regist = pd.concat(outputs, axis=0)  
        df_all_players_img_coords = pd.concat(outputs_all_players_img_coords, axis=0)
        print(time.time()-S)
        if p2p_file is not None:
            df_p2p_regist.to_csv(p2p_file, index=False)
            df_all_players_img_coords.to_csv(p2p_file.replace(".csv", "_img_coords.csv"), index=False)
    return df_p2p_regist, df_all_players_img_coords


## 以下2つの関数を make_features内で使用
df_p2p_regist_img_coordsのmerge追加。

In [ ]:
def p2p_matching_features(train_df, tr_tracking, phase):
    if phase == "test":
        p2p_file = None
    else:
        p2p_file = f"{cfg.KMAT_PATH}/output/p2p_registration_residuals.csv"
    tr_helmets, tr_video_metadata = load_helmet_meta(cfg, phase=phase)
    # registrationのoutputに全プレイヤのimg_coordsを追加。
    df_p2p_regist, df_p2p_regist_img_coords = p2p_registration_features(tr_tracking, tr_helmets, tr_video_metadata, p2p_file = p2p_file)
    
    df_p2p_regist["x_rel_position_offset_on_img"] = df_p2p_regist["x_position_offset_on_img"] / (df_p2p_regist["average_box_size"]+1e-7)
    df_p2p_regist["y_rel_position_offset_on_img"] = df_p2p_regist["y_position_offset_on_img"] / (df_p2p_regist["average_box_size"]+1e-7)

    merge_columns = ["game_play","step","nfl_player_id","x_position_offset_on_img","y_position_offset_on_img","x_rel_position_offset_on_img","y_rel_position_offset_on_img",
                    "p2p_registration_residual", 
                     "p2p_registration_residual_frame"
                    ]
    merge_columns_img_coords = ["game_play","step","nfl_player_id"] + ["img_coords_x", "img_coords_y"]
    
    for pair_no in [1]:
        train_df = pd.merge(train_df, df_p2p_regist.loc[df_p2p_regist["view"]=="Sideline", merge_columns], how="left", 
                     left_on=["game_play","step",f"nfl_player_id_{pair_no}"],
                     right_on=["game_play","step","nfl_player_id"])
        train_df.rename(columns={"x_position_offset_on_img":f"x_position_offset_on_img_Side_{pair_no}",
                                                                                            "y_position_offset_on_img":f"y_position_offset_on_img_Side_{pair_no}",
                                                                                            "x_rel_position_offset_on_img":f"x_rel_position_offset_on_img_Side_{pair_no}",
                                                                                            "y_rel_position_offset_on_img":f"y_rel_position_offset_on_img_Side_{pair_no}",
                                                                                            "p2p_registration_residual": f"nfl_player_id_{pair_no}_Side_residual",
                                                                                            "p2p_registration_residual_frame": f"nfl_player_id_{pair_no}_Side_residual_frame",
                                                                                            }, 
                        inplace=True)
        train_df.drop(columns=["nfl_player_id"], inplace=True)

        train_df = pd.merge(train_df, df_p2p_regist.loc[df_p2p_regist["view"]=="Endzone", merge_columns], how="left", 
                     left_on=["game_play","step",f"nfl_player_id_{pair_no}"],
                     right_on=["game_play","step","nfl_player_id"])
        train_df.rename(columns={"x_position_offset_on_img":f"x_position_offset_on_img_End_{pair_no}",
                                                                                            "y_position_offset_on_img":f"y_position_offset_on_img_End_{pair_no}",
                                                                                            "x_rel_position_offset_on_img":f"x_rel_position_offset_on_img_End_{pair_no}",
                                                                                            "y_rel_position_offset_on_img":f"y_rel_position_offset_on_img_End_{pair_no}",
                                                                                            "p2p_registration_residual":f"nfl_player_id_{pair_no}_End_residual",
                                                                                            "p2p_registration_residual_frame": f"nfl_player_id_{pair_no}_End_residual_frame",
                                                                                            },
                       inplace=True)
        train_df.drop(columns=["nfl_player_id"], inplace=True)
    
        """
        # image coords とりあえず外しときます。
        print(train_df.shape)
        train_df = pd.merge(train_df, df_p2p_regist_img_coords.loc[df_p2p_regist_img_coords["view"]=="Sideline", merge_columns_img_coords], how="left", 
                     left_on=["game_play","step",f"nfl_player_id_{pair_no}"],
                     right_on=["game_play","step","nfl_player_id"]).rename(columns={"img_coords_x":f"img_coords_x_Side{pair_no}",
                                                                                    "img_coords_y":f"img_coords_y_Side{pair_no}",
                                                                                    }).drop(columns=["nfl_player_id"])
        train_df = pd.merge(train_df, df_p2p_regist_img_coords.loc[df_p2p_regist_img_coords["view"]=="Endzone", merge_columns_img_coords], how="left", 
                     left_on=["game_play","step",f"nfl_player_id_{pair_no}"],
                     right_on=["game_play","step","nfl_player_id"]).rename(columns={"img_coords_x":f"img_coords_x_End{pair_no}",
                                                                                    "img_coords_y":f"img_coords_y_End{pair_no}",
                                                                                    }).drop(columns=["nfl_player_id"])
    
        print(train_df.shape)
        """
        
    train_df.head()
    return train_df

# 0219 update
def cnn_features(train_df, tr_tracking, phase, model_type="B"):
    tr_helmets, tr_video_metadata = load_helmet_meta(cfg, phase=phase) # 何回もロードしてマジすいません。
    
    if phase == "test":
        df_side, df_end, df_side_map, df_end_map = cnn_features_test(train_df, tr_tracking, tr_helmets, tr_video_metadata, model_type=model_type)
        # df_side, df_end, df_side_map, df_end_map = cnn_features_test_ensemble(train_df, tr_tracking, tr_helmets, tr_video_metadata, model_no_to_use, model_type=model_type)
    else:
        cnn_pred_path = f"{cfg.KMAT_PATH}/output/pred_model_{model_type}/"  
        # cnn_pred_path = "CROPCNN/"  
        df_side, df_end, df_side_map, df_end_map = cnn_features_val(train_df, tr_tracking, tr_helmets, tr_video_metadata, cnn_pred_path=cnn_pred_path, model_type=model_type)
  
    print(train_df.shape)

    # もともとのペアコンタクト予測
    train_df = pd.merge(train_df, 
                      df_side
                      [["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "cnn_pred_Sideline"]], 
                        on=["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"], how="left")
    train_df = pd.merge(train_df, 
                      df_end
                      [["game_play", "step", "nfl_player_id_1", "nfl_player_id_2","cnn_pred_Endzone"]], 
                      on=["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"], how="left")
    print(train_df.shape)

    del df_side, df_end
    gc.collect()

    camaro_df = pd.read_csv(cfg.CAMARO_DF1_PATH)
    camaro_df['camaro_pred'] = np.nan  # np.nanじゃないとroll feature作れなかった
    camaro_df['camaro_pred'] = camaro_df['camaro_pred'].astype(np.float32)
    camaro_df.loc[camaro_df['masks'], 'camaro_pred'] = camaro_df.loc[camaro_df['masks'], 'preds']
    merge_cols1 = ['game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2', 'camaro_pred']
    train_df = train_df.merge(camaro_df[merge_cols1], how='left')
    
    del camaro_df
    gc.collect()
    
    camaro_df2 = pd.read_csv(cfg.CAMARO_DF2_PATH)
    camaro_df2['camaro_pred2'] = np.nan  # np.nanじゃないとroll feature作れなかった
    camaro_df2['camaro_pred2'] = camaro_df2['camaro_pred2'].astype(np.float32)
    camaro_df2.loc[camaro_df2['masks'], 'camaro_pred2'] = camaro_df2.loc[camaro_df2['masks'], 'preds']
    merge_cols2 = ['game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2', 'camaro_pred2']
    train_df = train_df.merge(camaro_df2[merge_cols2], how='left')
    print('after merging camaro df:', train_df.shape)
    
    del camaro_df2
    gc.collect()
    
    df_side_map = reduce_dtype(df_side_map)
    df_end_map = reduce_dtype(df_end_map)

    # それ以外のシングルプレイヤー系の予測値。座標予測と、単独でのコンタクト予測
    new_columns = [] 
    for pid in [1,2]:
        train_df = pd.merge(train_df, 
                          df_end_map
                          [["game_play", "step", "nfl_player_id","pred_coords_i_Endzone","pred_coords_j_Endzone", "player_single_contacts_Endzone"]], 
                          left_on=["game_play", "step", f"nfl_player_id_{pid}"],
                          right_on=["game_play", "step", "nfl_player_id"], how="left")
        train_df.drop(columns=["nfl_player_id"], inplace=True)
        train_df.rename(
            columns={
                "pred_coords_i_Endzone": f"pred_coords_i_Endzone_pid{pid}",
                "pred_coords_j_Endzone": f"pred_coords_j_Endzone_pid{pid}", 
                "player_single_contacts_Endzone": f"player_single_contacts_Endzone_pid{pid}"
            },
            inplace=True)
        
        train_df = pd.merge(train_df, 
                          df_side_map
                          [["game_play", "step", "nfl_player_id","pred_coords_i_Sideline","pred_coords_j_Sideline", "player_single_contacts_Sideline"]], 
                          left_on=["game_play", "step", f"nfl_player_id_{pid}"],
                          right_on=["game_play", "step", "nfl_player_id"], how="left")
        train_df.drop(columns=["nfl_player_id"], inplace=True)
        train_df.rename(
            columns={
                "pred_coords_i_Sideline": f"pred_coords_i_Sideline_pid{pid}",
                "pred_coords_j_Sideline": f"pred_coords_j_Sideline_pid{pid}", 
                "player_single_contacts_Sideline": f"player_single_contacts_Sideline_pid{pid}"
            },
            inplace=True
        )
        
        new_columns += [f"pred_coords_i_Endzone_pid{pid}", f"pred_coords_j_Endzone_pid{pid}", f"player_single_contacts_Endzone_pid{pid}",
                       f"pred_coords_i_Sideline_pid{pid}", f"pred_coords_j_Sideline_pid{pid}", f"player_single_contacts_Sideline_pid{pid}"]
        gc.collect()
    
    del df_side_map, df_end_map
    gc.collect()
 
    # add some features
    for view in ["Endzone", "Sideline"]:
        train_df[f"pred_coords_delta_i_{view}"] = np.abs(train_df[f"pred_coords_i_{view}_pid1"] - train_df[f"pred_coords_i_{view}_pid2"])
        train_df[f"pred_coords_delta_j_{view}"] = np.abs(train_df[f"pred_coords_j_{view}_pid1"] - train_df[f"pred_coords_j_{view}_pid2"])
        train_df[f"pred_coords_dist_{view}"] = np.sqrt(train_df[f"pred_coords_delta_i_{view}"]**2 + train_df[f"pred_coords_delta_j_{view}"]**2)
        train_df[f"player_single_contacts_multiply_{view}"] = train_df[f"player_single_contacts_{view}_pid1"] * train_df[f"player_single_contacts_{view}_pid2"]
        train_df[f"pred_coords_distratio_{view}"] = train_df[f"pred_coords_dist_{view}"] / train_df["distance"]
        train_df[f"player_single_pair_relative_contacts_{view}_pid1"] = train_df[f"cnn_pred_{view}"] / train_df[f"player_single_contacts_{view}_pid1"]
        train_df[f"player_single_pair_relative_contacts_{view}_pid2"] = train_df[f"cnn_pred_{view}"] / train_df[f"player_single_contacts_{view}_pid2"]
        new_columns += [f"pred_coords_delta_i_{view}", f"pred_coords_delta_j_{view}", 
                       f"pred_coords_dist_{view}", f"player_single_contacts_multiply_{view}",
                       f"pred_coords_distratio_{view}", 
                       f"player_single_pair_relative_contacts_{view}_pid1", f"player_single_pair_relative_contacts_{view}_pid2"]
        
    # pair contact (different view
    train_df["player_single_contacts_multiply_EndSide_12"] = train_df["player_single_contacts_Endzone_pid1"] * train_df["player_single_contacts_Sideline_pid2"]
    train_df["player_single_contacts_multiply_EndSide_21"] = train_df["player_single_contacts_Endzone_pid2"] * train_df["player_single_contacts_Sideline_pid1"]
    
    # train_df[new_columns].to_csv("not_pair_predictions.csv", index=False)
    
    
    del train_df["datetime_ngs"]
    return train_df

def merege_2nd_candidate(df_inp, col_score, col_p1, col_p2, col_p1_err, col_p2_err, col_p1_err_base, col_p2_err_base, thresh_rate=1.0, suffix="A"):
    #"col_score": "cnn_pred_Sideline"
    #"col_p1": "nfl_player_id_1_Side_A"
    #"col_p2": "nfl_player_id_2"
    #"col_p1_err": "nfl_player_id_1_Side_A_residual"
    #"col_p2_err": "nfl_player_id_2_Side_residual"
    #"col_p1_err_base": "nfl_player_id_1_Side_residual"
    #"col_p2_err_base": "nfl_player_id_2_Side_residual"
     
    valid_pair = np.logical_and(df_inp[col_p1_err]<=(df_inp[col_p1_err_base]*thresh_rate), df_inp[col_p2_err]<=(df_inp[col_p2_err_base]*thresh_rate))

    df = df_inp.loc[valid_pair, ["game_play","step", col_p1, col_p2, col_score]].copy()
    #mask = np.logical_and()
    df = df.rename(columns={col_p1:"nfl_player_id_1", col_p2:"nfl_player_id_2",
                            col_score: f"{col_score}_{suffix}"
                           })
    df_inp = pd.merge(df_inp, df, on=["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"], how="left")
    return df_inp

def num_player_contact_with_other_player(df_inp):
    df = df_inp[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "cnn_pred_Sideline", "cnn_pred_Endzone"]].copy()
    df = pd.concat([df, df.rename(columns={"nfl_player_id_1":"nfl_player_id_2", "nfl_player_id_2":"nfl_player_id_1"})], axis=0)
    df = df.groupby(["game_play", "step", "nfl_player_id_1"])[["cnn_pred_Sideline", "cnn_pred_Endzone"]].max().reset_index()
    df = df.rename(columns={"cnn_pred_Sideline":"cnn_pred_Sideline_player_average", "cnn_pred_Endzone":"cnn_pred_Endzone_player_average", "nfl_player_id_1":"nfl_player_id"})

    df_p = df[df["nfl_player_id"]!=-1] # not ground
    df_g = df[df["nfl_player_id"]==-1] # ground
    
    df_inp = pd.merge(df_inp, df_p, left_on=["game_play", "step", "nfl_player_id_1"], right_on=["game_play", "step", "nfl_player_id"], how="left").drop(columns=["nfl_player_id"])
    df_inp = df_inp.rename(columns={"cnn_pred_Sideline_player_average":"cnn_pred_Sideline_player_average_1", "cnn_pred_Endzone_player_average":"cnn_pred_Endzone_player_average_1"})
    df_inp = pd.merge(df_inp, df_p, left_on=["game_play", "step", "nfl_player_id_2"], right_on=["game_play", "step", "nfl_player_id"], how="left").drop(columns=["nfl_player_id"])
    df_inp = df_inp.rename(columns={"cnn_pred_Sideline_player_average":"cnn_pred_Sideline_player_average_2", "cnn_pred_Endzone_player_average":"cnn_pred_Endzone_player_average_2"})
        
    df_inp["cnn_pred_Sideline_player_average_1"] = df_inp["cnn_pred_Sideline_player_average_1"] * (df_inp["cnn_pred_Sideline"]!=df_inp["cnn_pred_Sideline_player_average_1"])
    df_inp["cnn_pred_Endzone_player_average_1"] = df_inp["cnn_pred_Endzone_player_average_1"] * (df_inp["cnn_pred_Endzone"]!=df_inp["cnn_pred_Endzone_player_average_1"])
    df_inp["cnn_pred_Sideline_player_average_2"] = df_inp["cnn_pred_Sideline_player_average_2"] * (df_inp["cnn_pred_Sideline"]!=df_inp["cnn_pred_Sideline_player_average_2"])
    df_inp["cnn_pred_Endzone_player_average_2"] = df_inp["cnn_pred_Endzone_player_average_2"] * (df_inp["cnn_pred_Endzone"]!=df_inp["cnn_pred_Endzone_player_average_2"])

    df_inp = pd.merge(df_inp, df_g, left_on=["game_play", "step", "nfl_player_id_1"], right_on=["game_play", "step", "nfl_player_id"], how="left").drop(columns=["nfl_player_id"])
    df_inp = df_inp.rename(columns={"cnn_pred_Sideline_player_average":"cnn_pred_Sideline_ground_1", "cnn_pred_Endzone_player_average":"cnn_pred_Endzone_ground_1"})
    df_inp = pd.merge(df_inp, df_g, left_on=["game_play", "step", "nfl_player_id_2"], right_on=["game_play", "step", "nfl_player_id"], how="left").drop(columns=["nfl_player_id"])
    df_inp = df_inp.rename(columns={"cnn_pred_Sideline_player_average":"cnn_pred_Sideline_ground_2", "cnn_pred_Endzone_player_average":"cnn_pred_Endzone_ground_2"})
    return df_inp


class PairRollHolder:
    def __init__(self, window_size, all_players):
        self.window_size = window_size
        self.window_dev = int(window_size//2)
        self.pair_vals = {str(p1)+str(p2):[0]*window_size for p1 in all_players for p2 in all_players}
        self.pair_counts = {str(p1)+str(p2):[0]*window_size for p1 in all_players for p2 in all_players}
        self.roll_map = {}
        # self.roll_map_half = {}
        
    def ready_next(self):
        self.pair_vals = {key:val[1:]+[0] for key,val in self.pair_vals.items()}
        self.pair_counts = {key:val[1:]+[0] for key,val in self.pair_counts.items()}
        
    def add_if_not_nan(self, p1, p2, val):
        if not np.isnan(val):
            self.pair_vals[str(p1)+str(p2)][-1] = val
            self.pair_counts[str(p1)+str(p2)][-1] = 1
            self.pair_vals[str(p2)+str(p1)][-1] = val
            self.pair_counts[str(p2)+str(p1)][-1] = 1
            
    def end_of_step(self, step):
        self.roll_map.update({f"{step-self.window_dev}_" + key : sum(val)/(sum(self.pair_counts[key])+1e-7) for key,val in self.pair_vals.items()})
        # self.roll_map_half.update({f"{step-self.window_dev}_half_" + key : sum(val[:self.window_dev])/(sum(self.pair_counts[key][:self.window_dev])+1e-7) for key,val in self.pair_vals.items()})
        
        
def pair_step_roll_features_old(train_df, window_sizes=[5,11,21], columns_to_roll=['cnn_pred_Sideline', 'cnn_pred_Endzone', 'camaro_pred', 'camaro_pred2']):
    """
    game_play, pair(player1, player2), でstep方向にroll (現状average。ガウス重みがベター？)とる。
    画像系特徴の 予測欠損補間が主目的。特にGround接触時はヘルメットが隠れやすく、画像からの予測が存在しないところがあるはず。
    """
    df_roll_features = []
    c=0
    num_gp = len(train_df["game_play"].unique())
    feature_names = []
    for window_size in window_sizes:
        feature_names += [f"{column}_roll{window_size}" for column in columns_to_roll]
    for gp, df_gp in train_df.groupby("game_play"):
        print(f"\r{c} / {num_gp} game_play", end="")
        c+=1
        p1_uniq = list(df_gp["nfl_player_id_1"].unique())
        p2_uniq = list(df_gp["nfl_player_id_2"].unique())
        step_uniq = list(df_gp["step"].unique())
        df_gp["key"] = df_gp["step"].astype(str) + "_" + df_gp["nfl_player_id_1"].astype(str) + df_gp["nfl_player_id_2"].astype(str)
        min_step = np.min(step_uniq)
        max_step = np.max(step_uniq)
        all_players = list(set(p1_uniq + p2_uniq))
        num_player = len(all_players)
        num_step = int(1 + (max_step - min_step))
        dev = max(window_sizes)//2
        roll_feature_holders = {w: [PairRollHolder(w, all_players) for _ in columns_to_roll] for w in window_sizes}

        for step in range(min_step, max_step+1+dev):
            df_gp_step = df_gp[df_gp["step"]==step]
            p1s = df_gp_step["nfl_player_id_1"].values
            p2s = df_gp_step["nfl_player_id_2"].values
            values = [df_gp_step[column].values for column in columns_to_roll]
            for w, holders in roll_feature_holders.items():
                for holder in holders:
                    holder.ready_next() 
            for p1p2values in zip(p1s, p2s, *values):
                p1 = p1p2values[0]
                p2 = p1p2values[1]
                each_val = p1p2values[2:]
                for w, holders in roll_feature_holders.items():
                    for holder, v in zip(holders, each_val):
                        holder.add_if_not_nan(p1,p2,v)
            for w, holders in roll_feature_holders.items():
                for holder in holders:
                    holder.end_of_step(step) 
        for window, holders in roll_feature_holders.items():
            for column, holder in zip(columns_to_roll, holders):
                df_gp[f"{column}_roll{window}"] = df_gp["key"].map(holder.roll_map)
        df_roll_features.append(df_gp[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"]+feature_names])
        
    df_roll_features = pd.concat(df_roll_features, axis=0)
    # merge遅いし重いし良くないかも…
    train_df = pd.merge(train_df, df_roll_features, how="left", on=["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"])
    return train_df


# hata's version
def pair_step_roll_features(train_df, 
                            window_sizes=[5,11,21],
                            columns_to_roll= ['cnn_pred_Sideline', 'cnn_pred_Endzone', 'camaro_pred', 'camaro_pred2'],
                            full_sample_df=None):
    if full_sample_df is None:
        full_sample_df = train_df
        # raise NotImplementedError('未実装.元のデータを呼び出しても良い')

    merged_columns = ['game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2', 'distance']
    _tmp = pd.merge(full_sample_df[merged_columns], train_df[merged_columns+columns_to_roll],
                    on=merged_columns,
                   how='left'
                   )
    _tmp["distance"] < 3
    
    '''
    #nfl_player_idでスワップしたデータがテストデータに含まれていたら嫌なのでスワップしておく
    _tmp2 = _tmp.copy()
    _tmp2['nfl_player_id_1_2'] = _tmp2['nfl_player_id_1']
    _tmp2['nfl_player_id_1'] =_tmp2['nfl_player_id_2']
    _tmp2['nfl_player_id_2'] =_tmp2['nfl_player_id_1_2']
    del _tmp2['nfl_player_id_1_2']
    _tmp = pd.concat([_tmp, _tmp2], axis=0)
    '''
    # rolling featureを計算する。
    for window in window_sizes:
        renamed_columns = [f'{c}_roll{window}' for c in columns_to_roll]
        data = (_tmp
                .groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])[columns_to_roll]
                .rolling(window, center=True, min_periods=1)
                .mean()
                .reset_index()
                .rename(columns={c:f'{c}_roll{window}' for c in columns_to_roll})
                .sort_values('level_3')
                .reset_index(drop=True))
        
        for column in renamed_columns:
            _tmp[column] = data[column].fillna(0.0)
        merged_df = _tmp[merged_columns+[f'{c}_roll{window}' for c in columns_to_roll]]

        train_df = pd.merge(train_df, merged_df, on=merged_columns, how='left')

    return train_df


def pair_step_roll_features_for_nn_pred(train_df, 
                            window_sizes=[5,11,21],
                            columns_to_roll= ['cnn_pred_Sideline', 'cnn_pred_Endzone', 'camaro_pred', 'camaro_pred2'],
                            full_sample_df=None):
    """
    fill dist > threshold(3?) -> zero
    dist < 3 but no helmet detected -> nan
    """
    if full_sample_df is None:
        full_sample_df = train_df
        # raise NotImplementedError('未実装.元のデータを呼び出しても良い')

    merged_columns = ['game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2']
    isna_columns = [c+"_isna" for c in columns_to_roll]
    train_df[isna_columns] = train_df[columns_to_roll].isna()
    _tmp = pd.merge(full_sample_df[merged_columns], 
                    train_df[merged_columns+columns_to_roll+isna_columns],
                    on=merged_columns,
                   how='left'
                   )
    _tmp[columns_to_roll] = _tmp[columns_to_roll].fillna(0.0)
    for c_f, c_nan in zip(columns_to_roll, isna_columns):
        _tmp.loc[c_nan, c_f] = np.nan
    _tmp  = _tmp.drop(columns=isna_columns)
    '''
    #nfl_player_idでスワップしたデータがテストデータに含まれていたら嫌なのでスワップしておく
    _tmp2 = _tmp.copy()
    _tmp2['nfl_player_id_1_2'] = _tmp2['nfl_player_id_1']
    _tmp2['nfl_player_id_1'] =_tmp2['nfl_player_id_2']
    _tmp2['nfl_player_id_2'] =_tmp2['nfl_player_id_1_2']
    del _tmp2['nfl_player_id_1_2']
    _tmp = pd.concat([_tmp, _tmp2], axis=0)
    '''
    # rolling featureを計算する。
    for window in window_sizes:
        renamed_columns = [f'{c}_roll{window}' for c in columns_to_roll]
        data = (_tmp
                .groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])[columns_to_roll]
                .rolling(window, center=True, min_periods=1)
                .mean()
                .reset_index()
                .rename(columns={c:f'{c}_roll{window}' for c in columns_to_roll})
                .sort_values('level_3')
                .reset_index(drop=True))
        
        for column in renamed_columns:
            _tmp[column] = data[column].fillna(0.0)
        merged_df = _tmp[merged_columns+[f'{c}_roll{window}' for c in columns_to_roll]]
        train_df = pd.merge(train_df, merged_df, on=merged_columns, how='left')
    return train_df


def bbox_std_overlap_feature(df):
    for view in ["Sideline", "Endzone"]:
        xc1 = df[f"bbox_center_x_{view}_1"]
        yc1 = df[f"bbox_center_y_{view}_1"]
        xc2 = df[f"bbox_center_x_{view}_2"]
        yc2 = df[f"bbox_center_y_{view}_2"]
        w1 = df[f"width_{view}_1"]
        h1 = df[f"height_{view}_1"]
        w2 = df[f"width_{view}_2"]
        h2 = df[f"height_{view}_2"]

        df[f"bbox_x_std_overlap_{view}"] = (np.minimum(xc1+w1/2, xc2+w2/2) - np.maximum(xc1-w1/2, xc2-w2/2)) / (w1 + w2)
        df[f"bbox_y_std_overlap_{view}"] = (np.minimum(yc1+h1/2, yc2+h2/2) - np.maximum(yc1-h1/2, yc2-h2/2)) / (h1 + h2)

    return df


def interceptor_feature(df):
    # 1-2の間に別のプレイヤー(3)がいるかどうかを計算する。以下のどちらかに該当するケースを抽出。
    # - 角3-1-2が60度以下で、距離1-2より距離1-3のほうが短い場合
    # - 角3-2-1が60度以下で、距離2-1より距離2-3のほうが短い場合
    # 複数が該当する場合は角度が小さいものを優先する
    dy = df["y_position_2"] - df["y_position_1"]
    dx = df["x_position_2"] - df["x_position_1"]

    angle_th = 60

    df["angle_dxdy"] = np.rad2deg(np.arctan2(dy, dx))

    angles = df[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "angle_dxdy", "distance"]].copy()
    angles = angles[angles["nfl_player_id_2"] != -1]

    def negate_angle(s):
        s = s + 180
        s.loc[s > 180] = s.loc[s > 180] - 360
        return s

    angles_ = angles.copy()
    angles_.columns = ["game_play", "step", "nfl_player_id_2", "nfl_player_id_1", "angle_dxdy", "distance"]
    angles_["angle_dxdy"] = negate_angle(angles_["angle_dxdy"])

    angles = pd.concat([angles, angles_[angles.columns]]).reset_index(drop=True)

    angles_triplet = pd.merge(angles, angles, left_on=["game_play", "step", "nfl_player_id_2"], right_on=["game_play", "step", "nfl_player_id_1"], how="left")

    del angles_triplet["nfl_player_id_1_y"]
    angles_triplet.columns = ["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "angle_2to1", "distance_2to1", "nfl_player_id_3", "angle_2to3", "distance_2to3"]
    angles_triplet["angle_2to1"] = negate_angle(angles_triplet["angle_2to1"])
    angles_triplet = angles_triplet[angles_triplet["nfl_player_id_1"] != angles_triplet["nfl_player_id_3"]]

    angles_triplet["angle_123"] = angle_diff(angles_triplet["angle_2to1"], angles_triplet["angle_2to3"])

    interceptors = angles_triplet[(angles_triplet["distance_2to3"] <= angles_triplet["distance_2to1"]) & (angles_triplet["angle_123"] <= angle_th)].sort_values(by="angle_123").drop_duplicates(subset=["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"])

    interceptor_player2 = interceptors[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "distance_2to3", "angle_123", "nfl_player_id_3"]].copy()
    interceptor_player2.columns = ["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "distance_of_interceptor_2", "angle_interceptor_2", "nfl_player_id_interceptor_2"]

    interceptor_player1 = interceptors[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "distance_2to3", "angle_123", "nfl_player_id_3"]].copy()
    interceptor_player1.columns = ["game_play", "step", "nfl_player_id_2", "nfl_player_id_1", "distance_of_interceptor_1", "angle_interceptor_1", "nfl_player_id_interceptor_1"]

    df = pd.merge(df, interceptor_player1, on=["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"], how="left")
    df = pd.merge(df, interceptor_player2, on=["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"], how="left")
    
    return df


make features にphase追加。for CNN

In [ ]:
def add_cnn_shift_feature(df:pd.DataFrame, shift_steps:List[int]=[-5, -3, -1, 1, 3, 5]) -> pd.DataFrame:
    for shift_step in shift_steps:
        df[f'cnn_pred_Sideline_shift_{shift_step}'] = (df
                                                       .sort_values('step')
                                                       .groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])['cnn_pred_Sideline']
                                                       .shift(shift_step).reset_index().sort_values('index').set_index('index'))
        df[f'cnn_pred_Sideline_diff_{shift_step}'] = df['cnn_pred_Sideline'] - df[f'cnn_pred_Sideline_shift_{shift_step}']
        df[f'cnn_pred_Endzone_shift_{shift_step}'] = (df
                                                       .sort_values('step')
                                                       .groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])['cnn_pred_Endzone']
                                                       .shift(shift_step).reset_index().sort_values('index').set_index('index'))
        df[f'cnn_pred_Endzone_diff_{shift_step}'] = df['cnn_pred_Endzone'] - df[f'cnn_pred_Endzone_shift_{shift_step}']
    return df


def add_cnn_agg_features(train_df):
    def g_con_around_feature(df_train, dist_thresh = 1.5, columns = ['cnn_pred_Sideline_roll11', 'cnn_pred_Endzone_roll11']):
        """
        周辺が倒れている場合、その人も倒れている。
        画像中の隠れているプレイヤに対するグラウンドコンタクト紐づけ　兼　アサインメントミスの補助
        """
        print(df_train.shape)    
        add_columns = [n+"_g_contact_around" for n in columns]
        df_train_g = df_train.loc[df_train["nfl_player_id_2"]==-1, ["game_play", "step", "nfl_player_id_1"]+columns].rename(columns={c:a for c,a in zip(columns, add_columns)})
        p_pairs = df_train.loc[df_train["nfl_player_id_2"]!=-1, ["game_play", "step", "nfl_player_id_1", "nfl_player_id_2", "distance"]]
        p_pairs = p_pairs[p_pairs["distance"] < dist_thresh]
        p_pairs = pd.concat([p_pairs, p_pairs.rename(columns={"nfl_player_id_1": "nfl_player_id_2", "nfl_player_id_2":"nfl_player_id_1"})], axis=0)
        p_pairs = pd.merge(p_pairs, 
                          df_train_g, 
                          on=["game_play", "step", "nfl_player_id_1"], how="left")
        # 周辺プレイヤの地面コンタクト(自分以外)をとりあえずsumる。
        p_pairs = p_pairs.groupby(["game_play", "step", "nfl_player_id_2"])[add_columns].sum().reset_index()#("sum", "mean")
        p_pairs = p_pairs.rename(columns={"nfl_player_id_2":"nfl_player_id_1"})

        df_train = pd.merge(df_train, 
                          p_pairs, 
                          on=["game_play", "step", "nfl_player_id_1"], how="left")

        # df_train.loc[df_train["nfl_player_id_2"]!=-1, add_columns] = np.nan

        return df_train

    def g_conact_as_condition(df_train, score_columns=['cnn_pred_Sideline_roll5', 'cnn_pred_Endzone_roll5'], dist_ratio=1.):
        """
        同じ状態(高さ姿勢)のものはコンタクトしやすい
        グラウンドコンタクトの発生状態を高さ姿勢と考えてペア間の高さ姿勢を比較する
        あわせて、疑似的な距離を算出する。
        """
        print(df_train.shape)    
        temp_columns = [n+"_as_g_contact_cond" for n in score_columns]
        dev_columns = [n+"_dev_as_g_contact_cond" for n in score_columns]
        dist_columns = [n+"_dist_as_g_contact_cond" for n in score_columns]

        df_train_g = df_train.loc[df_train["nfl_player_id_2"]==-1, ["game_play", "step", "nfl_player_id_1"]+score_columns].rename(columns={c:a for c,a in zip(score_columns, temp_columns)})
        
        df_train = pd.merge(df_train, 
                          df_train_g, 
                          on=["game_play", "step", "nfl_player_id_1"], how="left").rename(columns={c: c+"_1" for c in temp_columns})
        df_train = pd.merge(df_train, 
                          df_train_g.rename(columns={"nfl_player_id_1": "nfl_player_id_2"}), 
                          on=["game_play", "step", "nfl_player_id_2"], how="left").rename(columns={c: c+"_2" for c in temp_columns})


        for dist_c, dev_c, temp_c in zip(dist_columns, dev_columns, temp_columns):
            df_train[dev_c] = np.abs(df_train[temp_c+"_2"] - df_train[temp_c+"_1"])
            df_train[dist_c] = np.sqrt((df_train[dev_c]*dist_ratio)**2 + df_train["distance"]**2)
            
        df_train = df_train.drop(columns=[c+"_1" for c in temp_columns] + [c+"_2" for c in temp_columns])

        return df_train
    
    def p_con_shift_feature(df_train, step_offset = 5, score_columns = ['cnn_pred_Sideline_roll11', 'cnn_pred_Endzone_roll11']):
        """
        過去にコンタクトがあった人は再度何かしらのコンタクト(特に地面と?)することが多い。
        (逆に地面コンタクトしている人はさかのぼると誰かにコンタクトしていると思う。TODO?)
        """
        print(df_train.shape)    
        add_columns = [n+f"_p_contact_past{step_offset}" for n in score_columns]
        df_train_p = df_train.loc[df_train["nfl_player_id_2"]!=-1, ["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"]+score_columns].rename(columns={c:a for c,a in zip(score_columns, add_columns)})
        df_train_p = pd.concat([df_train_p, df_train_p.rename(columns={"nfl_player_id_1": "nfl_player_id_2", "nfl_player_id_2":"nfl_player_id_1"})], axis=0)
        df_train_p = df_train_p.groupby(["game_play", "step", "nfl_player_id_1"])[add_columns].sum().reset_index()
        df_train_p["step"] = df_train_p["step"] + step_offset
        
        df_train = pd.merge(df_train, 
                          df_train_p, 
                          on=["game_play", "step", "nfl_player_id_1"], how="left").rename(columns={c: c+"_1" for c in add_columns})
        df_train = pd.merge(df_train, 
                          df_train_p.rename(columns={"nfl_player_id_1": "nfl_player_id_2"}), 
                          on=["game_play", "step", "nfl_player_id_2"], how="left").rename(columns={c: c+"_2" for c in add_columns})

        return df_train
    
    train_df = g_con_around_feature(train_df, dist_thresh = 1.5, columns = ['cnn_pred_Sideline_roll11', 'cnn_pred_Endzone_roll11', 'camaro_pred_roll11', 'camaro_pred2_roll11'])
    train_df = g_con_around_feature(train_df, dist_thresh = 0.75, columns = ['cnn_pred_Sideline_roll5', 'cnn_pred_Endzone_roll5', 'camaro_pred_roll5', 'camaro_pred2_roll5'])
    train_df = p_con_shift_feature(train_df, step_offset = 5, score_columns = ['cnn_pred_Sideline_roll5', 'cnn_pred_Endzone_roll5', 'camaro_pred_roll5', 'camaro_pred2_roll5'])
    train_df = p_con_shift_feature(train_df, step_offset = -5, score_columns = ['cnn_pred_Sideline_roll5', 'cnn_pred_Endzone_roll5', 'camaro_pred_roll5', 'camaro_pred2_roll5'])
    #train_df = g_conact_as_condition(train_df, score_columns = ['cnn_pred_Sideline_roll11', 'cnn_pred_Endzone_roll11'])
    return train_df


def agg_cnn_feature(df:pd.DataFrame) -> pd.DataFrame:
    for column in ['cnn_pred_Sideline', 'cnn_pred_Endzone', 'camaro_pred', 'camaro_pred2']:
        for agg in ['max', 'min', 'std']:
            df[f"{column}_{agg}_pair"] = df.groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])[column].transform(agg)
            df[f"{column}_{agg}_step"] = df.groupby(['game_play', 'step'])[column].transform(agg)
    return df

def add_distance_agg_feature(df):
    for shift in [1, -1]:
        df[f'distance_shift{shift}']=df.groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])['distance'].shift(shift)
    
    for roll in [5, 11, 21]:
        df[f'distance_window{roll}'] = (df.groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])['distance']
                                        .rolling(roll)
                                        .mean()
                                        .reset_index()
                                        .sort_values('level_3')
                                        .set_index('level_3')
                                        .rename(columns={'distance':f'distance_window{roll}'})[f'distance_window{roll}'])
    
    return df

In [ ]:
# phase "train" or "test"を追加させてください。
def make_features(df, tracking, phase, cnn_features_model_type="B"):
    with timer("merge"):
        tracking = tracking_prep(tracking)
        train_df = merge_cols(
            df,
            tracking,
            [
                "team", "position", "x_position", "y_position", 
                "speed", "distance", "direction", "orientation", "acceleration", 
                "sa",
                #"direction_p1_diff", "direction_m1_diff",
                #"orientation_p1_diff", "orientation_m1_diff",
                #"distance_p1", "distance_m1"
            ]
        )
        # Subsequent serialization in feather format replaces np.nan with None. 
        # For consistency between going through serialization and not, nan is converted here in advance.
        train_df["position_2"] = train_df["position_2"].replace({np.nan: None})

    print(train_df.shape)
    
    with timer("tracking_agg_features"):
        train_df = make_basic_features(train_df) 
        
        # この辺は全サンプルで計算
        train_df = create_bbox_features(train_df)
        # train_df = bbox_distance_around_player(train_df)
        train_df = add_distance_agg_feature(train_df)
        train_df = distance_around_player(train_df, True)
        train_df = second_nearest_distance(train_df, tracking, "1")
        train_df = second_nearest_distance(train_df, tracking, "2")
        train_df = reduce_dtype(train_df)
        gc.collect()

    with timer("cnn_features"):
        train_df = cnn_features(train_df, tracking, phase=phase, model_type=cnn_features_model_type)
        train_df = reduce_dtype(train_df)
        gc.collect()  

    with timer("pair_step_roll_features_for_nn_pred"):
        train_df = pair_step_roll_features_for_nn_pred(train_df, window_sizes = [5,11,21], 
                                           columns_to_roll = ['cnn_pred_Sideline', 'cnn_pred_Endzone', 'camaro_pred', 'camaro_pred2'],
                                           full_sample_df=train_df)
        train_df = reduce_dtype(train_df)
        gc.collect()

    with timer("sampling"):
        # メモリ対策。分けてみる
        is_hard_sample = np.logical_or(train_df["distance"]<=3, train_df["nfl_player_id_2"]==-1)
        # full_sample_df = train_df[['game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2']].copy()
        train_df = train_df[is_hard_sample]

    with timer("p2p_matching_features"):
        # distance 使用したいのでbasic直後に画像系特徴。
        train_df = p2p_matching_features(train_df, tracking, phase=phase)
        train_df = reduce_dtype(train_df)
        
        # p2pでとってきた特徴量のrollなので、p2pの後にやる
        train_df = pair_step_roll_features(train_df, window_sizes = [5,11,21], 
                                           columns_to_roll = ['x_rel_position_offset_on_img_End_1', 'y_rel_position_offset_on_img_Side_1'],
                                           full_sample_df=train_df)
        gc.collect()

    with timer("cnn-p2p"):
        # 追加
        train_df = reduce_dtype(train_df)
        train_df = add_cnn_agg_features(train_df)
        train_df = add_cnn_shift_feature(train_df)
        train_df = agg_cnn_feature(train_df)

        train_df = step_feature(train_df, tracking)
        train_df = tracking_agg_features(train_df, tracking)
        train_df = t0_feature(train_df, tracking)
        train_df = reduce_dtype(train_df)
        train_df = distance_around_player(train_df)
        train_df = aspect_ratio_feature(train_df, False)
        train_df = misc_features_after_agg(train_df)
        train_df = shift_of_player(train_df, tracking, [-5, 5, 10], add_diff=True, player_id="1")
        train_df = shift_of_player(train_df, tracking, [-5, 5], add_diff=True, player_id="2")
        train_df = bbox_std_overlap_feature(train_df)

        train_df = interceptor_feature(train_df)
        train_df = reduce_dtype(train_df)
        
        # TODO temporary!
        for view in ["Sideline", "Endzone"]:
            train_df[f"std_bbox_size_{view}"] = np.sqrt(train_df[f"width_{view}_mean"] * train_df[f"height_{view}_mean"])
            del train_df[f"width_{view}_mean"]
            del train_df[f"height_{view}_mean"]
        
        gc.collect()
        
    print(train_df.shape)
    print(train_df.columns.tolist())
    return train_df, is_hard_sample

# モデリング

## LightGBM

In [ ]:

def predict_lgbm(booster: Union[lgb.Booster, lgb.CVBooster, xgb.Booster], X: np.ndarray, init_score=None):
    try:
        y_pred = booster.predict(X, raw_score=init_score is not None)
        if init_score is not None:
            y_pred = special.expit(init_score + y_pred)

        return y_pred
    except TypeError:
        # 例外をキャッチすることで強引にxgboostを分岐する
        if isinstance(booster, xgb.Booster):
            feature_names = booster.feature_names
        else:
            feature_names = booster.boosters[0].feature_names
        
        return booster.predict(xgb.DMatrix(X, feature_names=feature_names))



def make_oof(cvboosters, X_train: np.ndarray, y_train: pd.Series, split, init_score=None):
    oof = np.zeros(len(X_train))

    for booster, (idx_train, idx_valid) in zip(cvboosters, split):
        init_score_fold = init_score[idx_valid] if init_score is not None and isinstance(init_score, np.ndarray) else init_score
        y_pred = predict_lgbm(booster, X_train[idx_valid], init_score_fold)
        oof[idx_valid] = y_pred
        print(f"{roc_auc_score(y_train.iloc[idx_valid].values, y_pred)}")
        
    return oof


def plot_importance(cvbooster, figsize=(12, 20)):
    raw_importances = cvbooster.feature_importance(importance_type='gain')
    feature_name = cvbooster.boosters[0].feature_name()
    importance_df = pd.DataFrame(data=raw_importances,
                                 columns=feature_name)
    # order by average importance across folds
    sorted_indices = importance_df.mean(axis=0).sort_values(ascending=False).index
    sorted_importance_df = importance_df.loc[:, sorted_indices]
    # plot top-n
    PLOT_TOP_N = 100
    plot_cols = sorted_importance_df.columns[:PLOT_TOP_N]
    _, ax = plt.subplots(figsize=figsize)
    ax.grid()
    ax.set_xscale('log')
    ax.set_ylabel('Feature')
    ax.set_xlabel('Importance')
    sns.boxplot(data=sorted_importance_df[plot_cols],
                orient='h',
                ax=ax)
    plt.show()

    
def search_best_threshold(y_true, y_pred):
    def func(x_list):
        score = matthews_corrcoef(y_true, y_pred>x_list[0])
        return -score

    x0 = [0.4]
    result = minimize(func, x0,  method="nelder-mead")
    
    return result.x[0]


def binarize_pred(y_pred, threshold, threshold2, threshold2_mask):
    return ~threshold2_mask*(y_pred>threshold)+threshold2_mask*(y_pred>threshold2)


def search_best_threshold_pair(y_true, y_pred, is_ground):
    def func(x_list):
        score = matthews_corrcoef(y_true, binarize_pred(y_pred, x_list[0], x_list[1], is_ground))
        return -score

    x0 = [0.5, 0.5]
    result = minimize(func, x0,  method="nelder-mead")

    return result.x[0], result.x[1]


def search_best_threshold_pair_optuna(y_true, y_pred, is_ground, n_trials=100):

    def objective(trial):
        th1 = trial.suggest_float('th1', 0.1, 0.6)
        th2 = trial.suggest_float('th2', 0.1, 0.6)
        score = matthews_corrcoef(y_true, binarize_pred(y_pred, th1, th2, is_ground))
        return -score

    study = optuna.create_study()  # Create a new study.
    study.optimize(objective, n_trials=n_trials)  # Invoke optimization of the objective function.
    return study.best_params["th1"], study.best_params["th2"]


def metrics(y_true, y_pred, threshold=None, threshold2=None, threshold2_mask=None):
    if threshold is None:
        threshold = search_best_threshold(y_true, y_pred)
        
    if threshold2 is not None and threshold2_mask is not None:
        return matthews_corrcoef(y_true, binarize_pred(y_pred, threshold, threshold2, threshold2_mask))
    else:
        return matthews_corrcoef(y_true, y_pred>threshold)

    
def make_x(train_df: pd.DataFrame, encoder: LabelEncoders, feature_names, use_existing_encoders=True):
    X_train = np.empty((len(train_df), len(feature_names)), dtype=np.float32)
    print(X_train.shape)

    for i, c in enumerate(feature_names):
        if train_df[c].dtype.name == "object":
            if use_existing_encoders:
                X_train[:, i] = encoder.transform_one(train_df[c])
            else:
                X_train[:, i] = encoder.fit_transform_one(train_df[c])
        else:
            X_train[:, i] = train_df[c]
            
    return X_train


def train_cv(train_df: pd.DataFrame, 
             non_feature_cols: List[str], 
             encoder: Optional[LabelEncoders] = None,
             lgb_params = None,
             seed = None,
             add_full_fold = True,
             debug = False):
    lgb_params = lgb_params or {
        "objective": "binary",
        "metric": "auc",
        "max_depth": -1,
        "num_leaves": 128,
        "feature_fraction": 0.3,
        "verbose": -1,
        "learning_rate": 0.02,
        "min_child_samples": 10,
        "min_child_weight": 5,
        "subsample_for_bin": 50000,
        #"reg_lambda": 1
    }
    if seed is not None:
        lgb_params["seed"] = seed

    is_ground = train_df["nfl_player_id_2"] == -1
    y_train = train_df["contact"]

    split_df = train_df[["game_play"]].copy()
    split_df["game"] = split_df["game_play"].str[:5].astype(int)
    split_df = pd.merge(split_df, split_defs, how="left")
    split = list(PredefinedSplit(split_df["fold"]).split())
    
    use_existing_encoders = encoder is not None
    encoder = encoder or LabelEncoders()

    with timer("make dataset"):
        # train_df.values.astype(np.float32)とかやるとOOMで死ぬので、箱を先に用意して値を入れていく
        feature_names = [c for c in train_df.columns if c not in non_feature_cols]

        X_train = make_x(train_df, encoder, feature_names, use_existing_encoders)

        gc.collect()
        print(f"features: {feature_names}")
        print(f"category: {list(encoder.encoders.keys())}")

        weight = None

        ds_train = lgb.Dataset(X_train, y_train,
                               feature_name=feature_names, 
                               weight=weight)
        gc.collect()

    with timer("lgb.cv"):
        if add_full_fold:
            split.append((np.arange(len(train_df)), np.arange(10)))

        ret = lgb.cv(lgb_params, ds_train, 
                     num_boost_round=50 if debug else 4000, 
                     folds=split,
                     return_cvbooster=True,
                     callbacks=[
                         #lgb.early_stopping(stopping_rounds=100, verbose=True),
                         lgb.log_evaluation(25)
                     ]) 

        for booster in ret["cvbooster"].boosters:
            booster.best_iteration = ret["cvbooster"].best_iteration

        del ds_train
        gc.collect()

    plot_importance(ret["cvbooster"])

    oof = make_oof(ret["cvbooster"].boosters[:4], X_train, y_train, split)

    # np.save("oof.npy", oof)

    print(is_ground.mean())
    
    if is_ground.mean() in [0.0, 1.0]:
        # g, non-gどちらかしかない。
        with timer("find best threshold"):
            threshold = search_best_threshold(y_train, oof)

        mcc = metrics(y_train, oof, threshold)
        auc = roc_auc_score(y_train, oof)
        print(f"threshold: {threshold:.5f}, mcc: {mcc:.5f}, auc: {auc:.5f}")

        return ret["cvbooster"], oof, encoder, threshold
    else:
        with timer("find best threshold"):
            #threshold = search_best_threshold(y_train, oof)
            threshold_p, threshold_g = search_best_threshold_pair_optuna(y_train, oof, is_ground)

        mcc = metrics(y_train, oof, threshold_p, threshold_g, is_ground)
        auc = roc_auc_score(y_train, oof)
        print(f"threshold: {threshold_p:.5f}, {threshold_g:.5f}, mcc: {mcc:.5f}, auc: {auc:.5f}")

        mcc_ground = metrics(y_train[is_ground], oof[is_ground], threshold_g)
        mcc_non_ground = metrics(y_train[~is_ground], oof[~is_ground], threshold_p)

        print(f"mcc(ground): {mcc_ground:.5f}, mcc(non-ground): {mcc_non_ground:.5f}")

        return ret["cvbooster"], oof, encoder, threshold_p, threshold_g

In [ ]:
class LGBMSerializer:
    def __init__(self,
                 booster: lgb.CVBooster,
                 encoders: LabelEncoders,
                 threshold_p: float,
                 threshold_g: float,
                 init_score: float = None,
                 treelite: bool = False,
                 feature_names = None):
        self.booster = booster
        self.encoders = encoders
        self.threshold_p = threshold_p
        self.threshold_g = threshold_g
        self.init_score = init_score
        self.is_xgb = isinstance(self.booster.boosters[0], xgb.Booster)
        self.treelite = treelite
        self.feature_names = feature_names
        
        if feature_names is None and not treelite:
            if self.is_xgb:
                self.feature_names = list(self.booster.boosters[0].feature_names)
            else:
                self.feature_names = self.booster.boosters[0].feature_name()

        self.encoders.patch()

    def to_file(self, filename: str):
        is_xgb = isinstance(self.booster.boosters[0], xgb.Booster)
        model = {
            "best_iteration": self.booster.best_iteration,
            "threshold_p": self.threshold_p,
            "threshold_g": self.threshold_g,
            "init_score": self.init_score,
            "is_xgb": is_xgb,
            "n_models": len(self.booster.boosters),
            "feature_names": self.feature_names
        }
        if is_xgb:
            for i, b in enumerate(self.booster.boosters):
                b.save_model(f'{filename}_xgb_{i}.json')
        else:
            model["boosters"] = [b.model_to_string() for b in self.booster.boosters]

        with open(f"{filename}_model.json", "w") as f:
            json.dump(model, f)

        with open(f"{filename}_encoder.bin", "wb") as f:
            pickle.dump(self.encoders, f)

    @classmethod
    def from_file(cls, filename: str, treelite: bool = False) -> "TrainedModel":

        with open(f"{filename}_model.json", "r") as f:
            model = json.load(f)

        cvbooster = lgb.CVBooster()

        feature_names = model.get("feature_names", [])

        if treelite:
            print("loading treelite...")
            lib_files = glob.glob(os.path.join(os.path.dirname(filename), "*.so"))
            assert len(lib_files) > 0, "treelite file not found!"
            for lib_file in lib_files:
                b = treelite_runtime.Predictor(lib_file, verbose=False)
                cvbooster.boosters.append(b)
        elif model.get("is_xgb"):
            print("loading native-xgb...")
            for i in range(model["n_models"]):
                b = xgb.Booster()
                b.load_model(f'{filename}_xgb_{i}.json')
                cvbooster.boosters.append(b)
            feature_names = list(b.feature_names)
        else:
            print("loading native-lgb...")
            cvbooster.boosters = [lgb.Booster(model_str=b) for b in model["boosters"]]
            cvbooster.best_iteration = model["best_iteration"]
            for b in cvbooster.boosters:
                b.best_iteration = cvbooster.best_iteration
            feature_names = b.feature_name()

        with open(f"{filename}_encoder.bin", "rb") as f:
            encoders = pickle.load(f)

        return cls(
            cvbooster,
            encoders,
            model["threshold_p"],
            model["threshold_g"],
            model.get("init_score", None),
            treelite=treelite,
            feature_names=feature_names)

    def predict(self, X: np.ndarray):
        if self.treelite:
            X_mat = treelite_runtime.DMatrix(X)
            predicted = []
            for b in self.booster.boosters:
                predicted.append(b.predict(X_mat))

            avg_pred = np.array(predicted).mean(axis=0)
        elif self.is_xgb:
            feature_names = self.booster.boosters[0].feature_names
            avg_pred = np.array(self.booster.predict(xgb.DMatrix(X, feature_names=feature_names))).mean(axis=0)
        else:
            avg_pred = np.array(predict_lgbm(self.booster, X, self.init_score)).mean(axis=0)

        return avg_pred



In [ ]:


# https://stackoverflow.com/questions/66681443/how-can-i-get-the-trained-model-from-xgboost-cv
class SaveBestModel(xgb.callback.TrainingCallback):
    def __init__(self, cvboosters):
        self._cvboosters = cvboosters

    def after_training(self, model):
        self._cvboosters[:] = [cvpack.bst for cvpack in model.cvfolds]
        return model


def train_cv_xgb(train_df: pd.DataFrame,
                 non_feature_cols: List[str],
                 encoder: Optional[LabelEncoders] = None,
                 xgb_params = None,
                 seed = None,
                 add_full_fold = True,
                 debug = False,
                 gpu = True):
    xgb_params = xgb_params or {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "max_depth": 9,
        "colsample_bytree": 0.3,
        "eta": 0.02,
        "min_child_weight": 5,
    }
    if seed is not None:
        xgb_params["seed"] = seed

    if gpu:
        xgb_params["gpu_id"] = 0
        xgb_params['tree_method'] = 'gpu_hist'

    is_ground = train_df["nfl_player_id_2"] == -1
    y_train = train_df["contact"]

    split_df = train_df[["game_play"]].copy()
    split_df["game"] = split_df["game_play"].str[:5].astype(int)
    split_df = pd.merge(split_df, split_defs, how="left")
    split = list(PredefinedSplit(split_df["fold"]).split())

    use_existing_encoders = encoder is not None
    encoder = encoder or LabelEncoders()

    with timer("make dataset"):
        feature_names = [c for c in train_df.columns if c not in non_feature_cols]

        X_train = make_x(train_df, encoder, feature_names, use_existing_encoders)

        gc.collect()

        weight = None

        ds_train = xgb.DMatrix(X_train, y_train, feature_names=feature_names, weight = weight)

        gc.collect()

    with timer("lgb.cv"):
        if add_full_fold:
            split.append((np.arange(len(train_df)), np.arange(200)))

        boosters = []

        xgb.cv(xgb_params, ds_train,
               num_boost_round=50 if debug else 4000,
               folds=split,
               verbose_eval=25,
               callbacks=[SaveBestModel(boosters)])

        cvboosters = lgb.CVBooster()
        cvboosters.boosters = boosters  # ただの箱として使うならxgb入れてもＯＫ

        #for booster in ret["cvbooster"].boosters:
        #    booster.best_iteration = ret["cvbooster"].best_iteration

        del ds_train
        gc.collect()

    # plot_importance_xgb(cvboosters)

    oof = make_oof(cvboosters.boosters[:4], X_train, y_train, split, init_score=None)

    # np.save("oof.npy", oof)

    print(is_ground.mean())

    if is_ground.mean() in [0.0, 1.0]:
        # g, non-gどちらかしかない。
        with timer("find best threshold"):
            threshold = search_best_threshold(y_train, oof)

        mcc = metrics(y_train, oof, threshold)
        auc = roc_auc_score(y_train, oof)
        print(f"threshold: {threshold:.5f}, mcc: {mcc:.5f}, auc: {auc:.5f}")

        return cvboosters, oof, encoder, threshold
    else:
        with timer("find best threshold"):
            threshold_p, threshold_g = search_best_threshold_pair_optuna(y_train, oof, is_ground)

        mcc = metrics(y_train, oof, threshold_p, threshold_g, is_ground)
        auc = roc_auc_score(y_train, oof)
        print(f"threshold: {threshold_p:.5f}, {threshold_g:.5f}, mcc: {mcc:.5f}, auc: {auc:.5f}")

        mcc_ground = metrics(y_train[is_ground], oof[is_ground], threshold_g)
        mcc_non_ground = metrics(y_train[~is_ground], oof[~is_ground], threshold_p)

        print(f"mcc(ground): {mcc_ground:.5f}, mcc(non-ground): {mcc_non_ground:.5f}")

        return cvboosters, oof, encoder, threshold_p, threshold_g

In [ ]:
non_feature_cols = [
    "contacgt_id",
    "game_play",
    "datetime",
    "step",
    "nfl_player_id_1",
    "nfl_player_id_2",
    "contact",
    "team_1",
    "team_2",
    "contact_id",
    #"position_1",
    #"position_2"
    #"direction_1",
    #"direction_2",
    "x_position_1",
    "x_position_2",
    "y_position_1",
    "y_position_2",
    "x_position_start_1",
    "x_position_start_2",
    "y_position_start_1",
    "y_position_start_2",

    "x_position_future5_1",
    "x_position_future5_2",
    "y_position_future5_1",
    "y_position_future5_2",
    "x_position_past5_1",
    "x_position_past5_2",
    "y_position_past5_1",
    "y_position_past5_2",
    "nfl_player_id_interceptor_1",
    "nfl_player_id_interceptor_2",

    #"orientation_past5_1",
    #"direction_past5_1",
    #"orientation_past5_2",
    #"direction_past5_2",
]


In [ ]:
def build_treelite(original_path: str, test_X: np.ndarray):
    dirname = os.path.dirname(original_path)
    original_model = LGBMSerializer.from_file(original_path)
    print(f"dirname: {dirname}")

    for i, b in enumerate(original_model.booster.boosters):
        print(f"compile model {i}")
        with timer("annotate"):
            if original_model.is_xgb:
                model = treelite.Model.from_xgboost(b)
            else:
                model = treelite.Model.from_lightgbm(b)

            dmat = treelite_runtime.DMatrix(data=test_X)
            annotator = treelite.Annotator()
            annotator.annotate_branch(model=model, dmat=dmat, verbose=True)
            annotator.save(path='annot.json')

        with timer("compile"):
            path = os.path.join(dirname, f'treelite_{i}.so')
            model.export_lib(toolchain='gcc',
                             libpath=path,
                             verbose=False,
                             params={'parallel_comp': 32, 'annotate_in': 'annot.json'},
                             nthread=32)
            print(f"model has been saved as {path}.")


def train_model(dst_directory: str, model_name: str, train_df: pd.DataFrame, non_feature_cols: List[str], is_xgb: bool = False, compile_treelite: bool = False, debug: bool = False):
    os.makedirs(f"{dst_directory}/{model_name}", exist_ok=True)
    if is_xgb:
        model_basename = f"{dst_directory}/{model_name}/model"
        cvbooster, oof, encoder, threshold_p, threshold_g = train_cv_xgb(train_df, non_feature_cols, debug=debug, gpu=cfg.USE_GPU, add_full_fold=False) # just forgot to add full-fold
        serializer = LGBMSerializer(cvbooster, encoder, threshold_p, threshold_g)
        serializer.to_file(f"{model_name}/model")
    else:
        model_basename = f"{dst_directory}/{model_name}/lgb"
        cvbooster, oof, encoder, threshold_p, threshold_g = train_cv(train_df, non_feature_cols, debug=debug, add_full_fold=True)
    serializer = LGBMSerializer(cvbooster, encoder, threshold_p, threshold_g)

    serializer.to_file(model_basename)
    np.save(f"{dst_directory}/{model_name}/oof.npy", oof)

    if compile_treelite:
        try:
            encoder.patch()
            test_X = make_x(train_df, encoder, serializer.feature_names)
            build_treelite(model_basename, test_X)
        except Exception:
            print("Failed to build treelite")
            print(traceback.format_exc())

    gc.collect()

    
with timer("make_features(K_mat-B)"):
    df_path = os.path.join(cfg.CACHE_DIR, "train_df_B.f")
    hard_sample_path = os.path.join(cfg.CACHE_DIR, "tr_is_hard_sample.f")

    if os.path.exists(df_path):
        train_df = pd.read_feather(df_path)
        tr_is_hard_sample = pd.read_feather(hard_sample_path)["is_hard_sample"]
    else:
        train_df, tr_is_hard_sample = make_features(train, tr_tracking, phase="train", cnn_features_model_type="B")
        gc.collect()

        train_df.to_feather(df_path)
        pd.DataFrame(tr_is_hard_sample, columns=["is_hard_sample"]).to_feather(hard_sample_path)


debug = False

# equivalent to dataset: nyanpn/nyanp-model-b-0227
with timer("train model(nyanp-b)"):
    train_model(cfg.INPUT_DIR, "nyanp-model-b-0227", train_df, non_feature_cols, is_xgb=False, debug=debug, compile_treelite=cfg.TREELITE_COMPILE)

# equivalent to dataset: nyanpn/nfl-kmat-only-2
with timer("train model(kmat-only)"):
    train_model(cfg.INPUT_DIR, "nfl-kmat-only-2", train_df, non_feature_cols + [c for c in train_df.columns if "camaro" in c], is_xgb=False, debug=debug, compile_treelite=cfg.TREELITE_COMPILE)

# equivalent to dataset: nyanpn/nfl-xgb-8030
with timer("train model(xgb-8030)"):
    train_model(cfg.INPUT_DIR, "nfl-xgb-8030", train_df, non_feature_cols, is_xgb=True, debug=debug, compile_treelite=cfg.TREELITE_COMPILE)

del train_df
gc.collect()


In [ ]:
with timer("make_features(K_mat-A)"):
    df_path = os.path.join(cfg.CACHE_DIR, "train_df_A.f")

    if os.path.exists(df_path):
        train_df = pd.read_feather(df_path)
    else:
        train_df, tr_is_hard_sample = make_features(train, tr_tracking, phase="train", cnn_features_model_type="A")
        gc.collect()

        train_df.to_feather(df_path)


# equivalent to dataset: nyanpn/nyanp-model-a-0227
with timer("train model(nyanp-a)"):
    train_model(cfg.INPUT_DIR, "nyanp-model-a-0227", train_df, non_feature_cols, is_xgb=False, debug=debug, compile_treelite=cfg.TREELITE_COMPILE)
